<a href="https://colab.research.google.com/github/nilesh3030/Natural-Language-Processing/blob/main/nlpdemystified_transformers_and_pretraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Processing Demystified | Transformers, Pre-training, and Transfer Learning
https://nlpdemystified.org<br>
https://github.com/nitinpunjabi/nlp-demystified<br><br>

Course module for this demo: https://www.nlpdemystified.org/course/transformers

**IMPORTANT**<br>
Enable **GPU acceleration** by going to *Runtime > Change Runtime Type*. Keep in mind that, on certain tiers, you're not guaranteed GPU access depending on usage history and current load.
<br><br>
Also, if you're running this in the cloud rather than a local Jupyter server on your machine, then the notebook will *timeout* after a period of inactivity.
<br><br>
Refer to this link on how to run Colab notebooks locally on your machine to avoid this issue:<br>
https://research.google.com/colaboratory/local-runtimes.html

In [1]:
!pip install BPEmb

import math
import numpy as np
import tensorflow as tf

from bpemb import BPEmb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.9 MB/s eta 0:00:00


# Transformers From Scratch

We'll build a transformer from scratch, layer-by-layer. We'll start with the **Multi-Head Self-Attention** layer since that's the most involved bit. Once we have that working, the rest of the model will look familiar if you've been following the course so far.

## Multi-Head Self-Attention

#### Scaled Dot Product Self-Attention


Inside each attention head is a **Scaled Dot Product Self-Attention** operation as we covered in the slides. Given *queries*, *keys*, and *values*, the operation returns a new "mix" of the values.

$$Attention(Q, K, V) = softmax(\frac{QK^T)}{\sqrt{d_k}})V$$

The following function implements this and also takes a mask to account for padding and for masking future tokens for decoding (i.e. **look-ahead mask**). We'll cover masking later in the notebook.

In [2]:
def scaled_dot_product_attention(query, key, value, mask=None):
  key_dim = tf.cast(tf.shape(key)[-1], tf.float32)
  scaled_scores = tf.matmul(query, key, transpose_b=True) / np.sqrt(key_dim)

  if mask is not None:
    scaled_scores = tf.where(mask==0, -np.inf, scaled_scores)

  softmax = tf.keras.layers.Softmax()
  weights = softmax(scaled_scores) 
  return tf.matmul(weights, value), weights

Suppose our *queries*, *keys*, and *values* are each a length of 3 with a dimension of 4.

In [3]:
seq_len = 3
embed_dim = 4

queries = np.random.rand(seq_len, embed_dim)
keys = np.random.rand(seq_len, embed_dim)
values = np.random.rand(seq_len, embed_dim)

print("Queries:\n", queries)

Queries:
 [[0.88001735 0.33120815 0.52169634 0.31047457]
 [0.52863677 0.04746922 0.74963699 0.17178154]
 [0.81243677 0.46569175 0.23329311 0.33453527]]


This would be the self-attention output and weights.

In [4]:
output, attn_weights = scaled_dot_product_attention(queries, keys, values)

print("Output\n", output, "\n")
print("Weights\n", attn_weights)

Output
 tf.Tensor(
[[0.53453815 0.95767295 0.32656214 0.2765804 ]
 [0.52731717 0.9572627  0.32758504 0.28030413]
 [0.5588641  0.9591112  0.32089698 0.2636807 ]], shape=(3, 4), dtype=float32) 

Weights
 tf.Tensor(
[[0.34246367 0.2649866  0.39254972]
 [0.34289053 0.25691313 0.40019637]
 [0.33815008 0.29444194 0.36740795]], shape=(3, 3), dtype=float32)


#### Generating queries, keys, and values for multiple heads.

Now that we have a way to calculate self-attention, let's actually generate the input *queries*, *keys*, and *values* for multiple heads.
<br><br>
In the slides (and in most references), each attention head had its <u>own separate</u> set of *query*, *key*, and *value* weights. Each weight matrix was of dimension $d\ x \ d/h$ where h was the number of heads. 

![](https://drive.google.com/uc?export=view&id=1SLWkHQgy4nQPFvvjG5_V8UTtpSAJ2zrr)

It's easier to understand things this way and we can certainly code it this way as well. But we can also "simulate" different heads with a single query matrix, single key matrix, and single value matrix.
<br><br>
We'll do both. First we'll create *query*, *key*, and *value* vectors using separate weights per head.
<br><br>
In the slides, we used an example of 12 dimensional embeddings processed by  three attentions heads, and we'll do the same here.

In [5]:
batch_size = 1
seq_len = 3
embed_dim = 12
num_heads = 3
head_dim = embed_dim // num_heads

print(f"Dimension of each head: {head_dim}")

Dimension of each head: 4


**Using separate weight matrices per head**

Suppose these are our input embeddings. Here we have a batch of 1 containing a sequence of length 3, with each element being a 12-dimensional embedding.

In [6]:
x = np.random.rand(batch_size, seq_len, embed_dim).round(1)
print("Input shape: ", x.shape, "\n")
print("Input:\n", x)

Input shape:  (1, 3, 12) 

Input:
 [[[0.6 0.6 1.  0.9 0.9 0.8 0.5 0.3 0.4 0.7 0.  0.3]
  [0.1 0.6 0.6 0.3 0.7 0.3 0.  0.4 0.4 0.  0.7 0.7]
  [1.  0.2 0.1 0.6 0.1 0.9 0.4 0.7 0.6 0.5 0.8 0.5]]]


We'll declare three sets of *query* weights (one for each head), three sets of *key* weights, and three sets of *value* weights. Remember each weight matrix should have a dimension of $\text{d}\ \text{x}\ \text{d/h}$.

In [7]:
# The query weights for each head.
wq0 = np.random.rand(embed_dim, head_dim).round(1)
wq1 = np.random.rand(embed_dim, head_dim).round(1)
wq2 = np.random.rand(embed_dim, head_dim).round(1)

# The key weights for each head. 
wk0 = np.random.rand(embed_dim, head_dim).round(1)
wk1 = np.random.rand(embed_dim, head_dim).round(1)
wk2 = np.random.rand(embed_dim, head_dim).round(1)

# The value weights for each head.
wv0 = np.random.rand(embed_dim, head_dim).round(1)
wv1 = np.random.rand(embed_dim, head_dim).round(1)
wv2 = np.random.rand(embed_dim, head_dim).round(1)

In [8]:
print("The three sets of query weights (one for each head):")
print("wq0:\n", wq0)
print("wq1:\n", wq1)
print("wq2:\n", wq1)

The three sets of query weights (one for each head):
wq0:
 [[0.2 0.2 0.3 0.5]
 [0.  0.7 0.3 0.1]
 [0.5 0.3 0.3 0.6]
 [0.7 0.3 0.2 0.4]
 [0.5 0.9 0.9 0.9]
 [0.3 0.9 0.3 0.2]
 [0.5 1.  0.9 0.8]
 [0.9 0.6 0.9 0.9]
 [0.5 0.6 0.2 0.5]
 [0.8 0.3 0.4 0.1]
 [0.2 0.1 0.1 0.7]
 [0.1 0.2 0.4 0.5]]
wq1:
 [[0.4 0.  0.1 0.4]
 [1.  0.2 0.3 0.3]
 [0.4 0.8 0.9 0.2]
 [0.3 0.3 0.3 0.2]
 [0.8 0.2 0.6 0.3]
 [0.6 0.7 0.6 0.8]
 [0.1 0.1 0.6 0.1]
 [0.6 0.2 0.3 0.5]
 [0.8 1.  0.8 0.4]
 [0.9 0.7 0.1 0.2]
 [0.4 0.2 0.1 0.5]
 [1.  0.7 0.9 0.2]]
wq2:
 [[0.4 0.  0.1 0.4]
 [1.  0.2 0.3 0.3]
 [0.4 0.8 0.9 0.2]
 [0.3 0.3 0.3 0.2]
 [0.8 0.2 0.6 0.3]
 [0.6 0.7 0.6 0.8]
 [0.1 0.1 0.6 0.1]
 [0.6 0.2 0.3 0.5]
 [0.8 1.  0.8 0.4]
 [0.9 0.7 0.1 0.2]
 [0.4 0.2 0.1 0.5]
 [1.  0.7 0.9 0.2]]


We'll generate our *queries*, *keys*, and *values* for each head by multiplying our input by the weights.

In [9]:
# Geneated queries, keys, and values for the first head.
q0 = np.dot(x, wq0)
k0 = np.dot(x, wk0)
v0 = np.dot(x, wv0)

# Geneated queries, keys, and values for the second head.
q1 = np.dot(x, wq1)
k1 = np.dot(x, wk1)
v1 = np.dot(x, wv1)

# Geneated queries, keys, and values for the third head.
q2 = np.dot(x, wq2)
k2 = np.dot(x, wk2)
v2 = np.dot(x, wv2)

These are the resulting *query*, *key*, and *value* vectors for the first head.

In [10]:
print("Q, K, and V for first head:\n")

print(f"q0 {q0.shape}:\n", q0, "\n")
print(f"k0 {k0.shape}:\n", k0, "\n")
print(f"v0 {v0.shape}:\n", v0)

Q, K, and V for first head:

q0 (1, 3, 4):
 [[[3.25 3.83 3.09 3.38]
  [1.74 2.3  1.96 2.68]
  [2.73 2.96 2.46 3.2 ]]] 

k0 (1, 3, 4):
 [[[3.41 3.51 3.71 3.67]
  [2.1  2.04 2.39 2.39]
  [3.34 2.28 3.34 2.98]]] 

v0 (1, 3, 4):
 [[[2.81 3.08 4.28 3.  ]
  [2.23 2.1  3.09 2.73]
  [2.99 2.91 3.81 3.09]]]


Now that we have our Q, K, V vectors, we can just pass them to our self-attention operation. Here we're calculating the output and attention weights for the first head.

In [11]:
out0, attn_weights0 = scaled_dot_product_attention(q0, k0, v0)

print("Output from first attention head: ", out0, "\n")
print("Attention weights from first head: ", attn_weights0)

Output from first attention head:  tf.Tensor(
[[[2.8125799 3.0774028 4.2729797 3.0012922]
  [2.8190746 3.0672562 4.248918  3.004592 ]
  [2.8151314 3.0744383 4.265335  3.002575 ]]], shape=(1, 3, 4), dtype=float32) 

Attention weights from first head:  tf.Tensor(
[[[9.85223293e-01 1.05027706e-04 1.46716805e-02]
  [9.38051820e-01 2.73137563e-03 5.92167974e-02]
  [9.69515204e-01 4.68343118e-04 3.00164670e-02]]], shape=(1, 3, 3), dtype=float32)


Here are the other two (attention weights are ignored).

In [12]:
out1, _ = scaled_dot_product_attention(q1, k1, v1)
out2, _ = scaled_dot_product_attention(q2, k2, v2)

print("Output from second attention head: ", out1, "\n")
print("Output from third attention head: ", out2,)

Output from second attention head:  tf.Tensor(
[[[3.6123753 3.4325714 2.8910663 4.1445055]
  [3.5966477 3.426169  2.848069  4.1049523]
  [3.591525  3.435305  2.804224  4.0682306]]], shape=(1, 3, 4), dtype=float32) 

Output from third attention head:  tf.Tensor(
[[[3.276435  2.7178073 3.2153256 3.2624912]
  [3.2268167 2.8030596 3.20815   3.2900314]
  [3.231248  2.79959   3.2109962 3.290989 ]]], shape=(1, 3, 4), dtype=float32)


As we covered in the slides, once we have each head's output, we concatenate them and then put them through a linear layer for further processing.

In [13]:
combined_out_a = np.concatenate((out0, out1, out2), axis=-1)
print(f"Combined output from all heads {combined_out_a.shape}:")
print(combined_out_a)

# The final step would be to run combined_out_a through a linear/dense layer 
# for further processing.

Combined output from all heads (1, 3, 12):
[[[2.8125799 3.0774028 4.2729797 3.0012922 3.6123753 3.4325714 2.8910663
   4.1445055 3.276435  2.7178073 3.2153256 3.2624912]
  [2.8190746 3.0672562 4.248918  3.004592  3.5966477 3.426169  2.848069
   4.1049523 3.2268167 2.8030596 3.20815   3.2900314]
  [2.8151314 3.0744383 4.265335  3.002575  3.591525  3.435305  2.804224
   4.0682306 3.231248  2.79959   3.2109962 3.290989 ]]]


So that's a complete run of **multi-head self-attention** using separate sets of weights per head.<br>

Let's now get the same thing done using a single query weight matrix, single key weight matrix, and single value weight matrix.<br><br>
These were our separate per-head query weights:

In [14]:
print("Query weights for first head: \n", wq0, "\n")
print("Query weights for second head: \n", wq1, "\n")
print("Query weights for third head: \n", wq2)

Query weights for first head: 
 [[0.2 0.2 0.3 0.5]
 [0.  0.7 0.3 0.1]
 [0.5 0.3 0.3 0.6]
 [0.7 0.3 0.2 0.4]
 [0.5 0.9 0.9 0.9]
 [0.3 0.9 0.3 0.2]
 [0.5 1.  0.9 0.8]
 [0.9 0.6 0.9 0.9]
 [0.5 0.6 0.2 0.5]
 [0.8 0.3 0.4 0.1]
 [0.2 0.1 0.1 0.7]
 [0.1 0.2 0.4 0.5]] 

Query weights for second head: 
 [[0.4 0.  0.1 0.4]
 [1.  0.2 0.3 0.3]
 [0.4 0.8 0.9 0.2]
 [0.3 0.3 0.3 0.2]
 [0.8 0.2 0.6 0.3]
 [0.6 0.7 0.6 0.8]
 [0.1 0.1 0.6 0.1]
 [0.6 0.2 0.3 0.5]
 [0.8 1.  0.8 0.4]
 [0.9 0.7 0.1 0.2]
 [0.4 0.2 0.1 0.5]
 [1.  0.7 0.9 0.2]] 

Query weights for third head: 
 [[0.6 0.  0.3 0.5]
 [0.8 0.5 0.9 0.5]
 [0.3 0.1 0.4 0.2]
 [0.9 0.8 0.7 0.7]
 [0.1 0.3 1.  0.6]
 [0.7 0.3 0.8 0.1]
 [0.3 0.9 0.3 0.2]
 [0.  0.7 0.4 0.5]
 [0.4 0.1 1.  0.5]
 [0.1 0.6 0.9 0.6]
 [0.6 0.4 0.1 0.3]
 [0.5 0.2 0.9 0.5]]


Suppose instead of declaring three separate query weight matrices, we had declared one. i.e. a single $d\ x\ d$ matrix. We're concatenating our per-head query weights here instead of declaring a new set of weights so that we get the same results.

In [15]:
wq = np.concatenate((wq0, wq1, wq2), axis=1)
print(f"Single query weight matrix {wq.shape}: \n", wq)

Single query weight matrix (12, 12): 
 [[0.2 0.2 0.3 0.5 0.4 0.  0.1 0.4 0.6 0.  0.3 0.5]
 [0.  0.7 0.3 0.1 1.  0.2 0.3 0.3 0.8 0.5 0.9 0.5]
 [0.5 0.3 0.3 0.6 0.4 0.8 0.9 0.2 0.3 0.1 0.4 0.2]
 [0.7 0.3 0.2 0.4 0.3 0.3 0.3 0.2 0.9 0.8 0.7 0.7]
 [0.5 0.9 0.9 0.9 0.8 0.2 0.6 0.3 0.1 0.3 1.  0.6]
 [0.3 0.9 0.3 0.2 0.6 0.7 0.6 0.8 0.7 0.3 0.8 0.1]
 [0.5 1.  0.9 0.8 0.1 0.1 0.6 0.1 0.3 0.9 0.3 0.2]
 [0.9 0.6 0.9 0.9 0.6 0.2 0.3 0.5 0.  0.7 0.4 0.5]
 [0.5 0.6 0.2 0.5 0.8 1.  0.8 0.4 0.4 0.1 1.  0.5]
 [0.8 0.3 0.4 0.1 0.9 0.7 0.1 0.2 0.1 0.6 0.9 0.6]
 [0.2 0.1 0.1 0.7 0.4 0.2 0.1 0.5 0.6 0.4 0.1 0.3]
 [0.1 0.2 0.4 0.5 1.  0.7 0.9 0.2 0.5 0.2 0.9 0.5]]


In the same vein, pretend we declared a single key weight matrix, and single value weight matrix.

In [16]:
wk = np.concatenate((wk0, wk1, wk2), axis=1)
wv = np.concatenate((wv0, wv1, wv2), axis=1)

print(f"Single key weight matrix {wk.shape}:\n", wk, "\n")
print(f"Single value weight matrix {wv.shape}:\n", wv)

Single key weight matrix (12, 12):
 [[0.8 0.1 0.6 0.5 0.3 0.4 0.1 1.  0.4 0.3 0.8 0.6]
 [0.3 0.9 0.4 0.9 0.7 0.1 0.5 0.4 0.8 0.3 0.6 0.6]
 [0.5 0.3 0.7 1.  0.3 0.9 0.5 0.3 0.2 0.3 1.  0. ]
 [0.5 1.  0.3 0.3 0.2 0.2 0.4 0.1 0.3 0.6 0.2 0.2]
 [0.3 0.6 0.6 0.2 0.1 0.4 0.9 0.1 0.3 0.5 0.2 0.8]
 [0.8 0.3 0.8 0.5 0.9 0.5 0.9 0.7 0.9 0.6 0.4 0.4]
 [0.3 0.5 0.5 0.5 0.6 0.6 0.4 0.9 0.6 0.4 0.7 0. ]
 [0.3 0.1 0.3 0.5 0.6 0.  0.8 0.8 0.6 0.7 0.8 0.6]
 [0.3 0.1 0.6 0.5 0.  0.2 0.2 0.1 0.6 0.1 0.9 0.8]
 [0.5 0.7 0.5 0.5 0.7 0.3 0.2 0.3 0.1 0.7 1.  1. ]
 [0.4 0.2 0.7 0.5 0.7 0.9 0.6 0.1 0.7 0.3 0.2 0.7]
 [0.6 0.4 0.1 0.1 0.9 1.  0.1 0.6 0.8 0.9 0.  0.4]] 

Single value weight matrix (12, 12):
 [[0.4 0.4 0.2 0.2 0.2 0.9 0.  0.2 0.5 0.5 0.7 0.6]
 [0.2 0.8 0.5 0.5 0.5 0.2 0.3 0.7 0.3 0.2 0.7 0.5]
 [0.2 0.8 0.4 0.2 0.  0.7 0.9 0.9 0.6 0.  0.3 0.4]
 [0.2 0.  0.3 0.6 0.6 0.5 0.8 0.6 0.6 0.4 0.8 0.1]
 [0.5 0.  1.  0.6 0.9 0.3 0.2 0.8 0.9 0.  0.3 0. ]
 [0.4 0.7 1.  0.  0.8 0.2 0.5 0.9 0.2 0.7 0.4 0.6]
 [0.8

Now we can calculate all our *queries*, *keys*, and *values* with three dot products.

In [17]:
q_s = np.dot(x, wq)
k_s = np.dot(x, wk)
v_s = np.dot(x, wv)

These are our resulting query vectors (we'll call them "combined queries"). How do we simulate different heads with this?

In [18]:
print(f"Query vectors using a single weight matrix {q_s.shape}:\n", q_s)

Query vectors using a single weight matrix (1, 3, 12):
 [[[3.25 3.83 3.09 3.38 4.19 3.14 3.48 2.27 3.13 2.81 4.86 3.07]
  [1.74 2.3  1.96 2.68 3.25 2.15 2.56 1.7  2.2  1.64 3.22 2.09]
  [2.73 2.96 2.46 3.2  3.65 2.59 2.54 2.58 3.11 2.52 3.74 2.71]]]


Somehow, we need to separate these vectors such they're treated like three separate sets by the self-attention operation.

In [19]:
print(q0, "\n")
print(q1, "\n")
print(q2)

[[[3.25 3.83 3.09 3.38]
  [1.74 2.3  1.96 2.68]
  [2.73 2.96 2.46 3.2 ]]] 

[[[4.19 3.14 3.48 2.27]
  [3.25 2.15 2.56 1.7 ]
  [3.65 2.59 2.54 2.58]]] 

[[[3.13 2.81 4.86 3.07]
  [2.2  1.64 3.22 2.09]
  [3.11 2.52 3.74 2.71]]]


Notice how each set of per-head queries looks like we took the combined queries, and chopped them vertically every four dimensions.
<br><br>
We can split our combined queries into $\text{d}\ \text{x}\ \text{d/h}$ heads using **reshape** and **transpose**.<br><br>
The first step is to *reshape* our combined queries from a shape of:<br>
(batch_size, seq_len, embed_dim)<br>

into a shape of<br>
 (batch_size, seq_len, num_heads, head_dim).
 <br>

 https://www.tensorflow.org/api_docs/python/tf/reshape

In [20]:
# Note: we can achieve the same thing by passing -1 instead of seq_len.
q_s_reshaped = tf.reshape(q_s, (batch_size, seq_len, num_heads, head_dim))
print(f"Combined queries: {q_s.shape}\n", q_s, "\n")
print(f"Reshaped into separate heads: {q_s_reshaped.shape}\n", q_s_reshaped)

Combined queries: (1, 3, 12)
 [[[3.25 3.83 3.09 3.38 4.19 3.14 3.48 2.27 3.13 2.81 4.86 3.07]
  [1.74 2.3  1.96 2.68 3.25 2.15 2.56 1.7  2.2  1.64 3.22 2.09]
  [2.73 2.96 2.46 3.2  3.65 2.59 2.54 2.58 3.11 2.52 3.74 2.71]]] 

Reshaped into separate heads: (1, 3, 3, 4)
 tf.Tensor(
[[[[3.25 3.83 3.09 3.38]
   [4.19 3.14 3.48 2.27]
   [3.13 2.81 4.86 3.07]]

  [[1.74 2.3  1.96 2.68]
   [3.25 2.15 2.56 1.7 ]
   [2.2  1.64 3.22 2.09]]

  [[2.73 2.96 2.46 3.2 ]
   [3.65 2.59 2.54 2.58]
   [3.11 2.52 3.74 2.71]]]], shape=(1, 3, 3, 4), dtype=float64)


At this point, we have our desired shape. The next step is to *transpose* it such that simulates vertically chopping our combined queries. By transposing, our matrix dimensions become:<br>
(batch_size, num_heads, seq_len, head_dim)<br>

https://www.tensorflow.org/api_docs/python/tf/transpose

In [21]:
q_s_transposed = tf.transpose(q_s_reshaped, perm=[0, 2, 1, 3]).numpy()
print(f"Queries transposed into \"separate\" heads {q_s_transposed.shape}:\n", 
      q_s_transposed)

Queries transposed into "separate" heads (1, 3, 3, 4):
 [[[[3.25 3.83 3.09 3.38]
   [1.74 2.3  1.96 2.68]
   [2.73 2.96 2.46 3.2 ]]

  [[4.19 3.14 3.48 2.27]
   [3.25 2.15 2.56 1.7 ]
   [3.65 2.59 2.54 2.58]]

  [[3.13 2.81 4.86 3.07]
   [2.2  1.64 3.22 2.09]
   [3.11 2.52 3.74 2.71]]]]


If we compare this against the separate per-head queries we calculated previously, we see the same result except we now have all our queries in a single matrix.

In [22]:
print("The separate per-head query matrices from before: ")
print(q0, "\n")
print(q1, "\n")
print(q2)

The separate per-head query matrices from before: 
[[[3.25 3.83 3.09 3.38]
  [1.74 2.3  1.96 2.68]
  [2.73 2.96 2.46 3.2 ]]] 

[[[4.19 3.14 3.48 2.27]
  [3.25 2.15 2.56 1.7 ]
  [3.65 2.59 2.54 2.58]]] 

[[[3.13 2.81 4.86 3.07]
  [2.2  1.64 3.22 2.09]
  [3.11 2.52 3.74 2.71]]]


Let's do the exact same thing with our combined keys and values.

In [23]:
k_s_transposed = tf.transpose(tf.reshape(k_s, (batch_size, -1, num_heads, head_dim)), perm=[0, 2, 1, 3]).numpy()
v_s_transposed = tf.transpose(tf.reshape(v_s, (batch_size, -1, num_heads, head_dim)), perm=[0, 2, 1, 3]).numpy()

print(f"Keys for all heads in a single matrix {k_s.shape}: \n", k_s_transposed, "\n")
print(f"Values for all heads in a single matrix {v_s.shape}: \n", v_s_transposed)

Keys for all heads in a single matrix (1, 3, 12): 
 [[[[3.41 3.51 3.71 3.67]
   [2.1  2.04 2.39 2.39]
   [3.34 2.28 3.34 2.98]]

  [[3.13 3.03 3.44 3.  ]
   [2.39 2.54 2.52 1.68]
   [3.43 2.85 2.86 3.32]]

  [[3.21 3.34 4.17 3.26]
   [2.74 2.26 2.18 2.49]
   [3.63 3.09 3.56 3.44]]]] 

Values for all heads in a single matrix (1, 3, 12): 
 [[[[2.81 3.08 4.28 3.  ]
   [2.23 2.1  3.09 2.73]
   [2.99 2.91 3.81 3.09]]

  [[3.73 3.41 3.4  4.59]
   [2.54 2.27 1.89 2.99]
   [3.46 3.47 2.21 3.55]]

  [[3.45 2.4  3.23 3.15]
   [2.57 1.84 2.   1.93]
   [2.85 3.5  3.18 3.54]]]]


Set up this way, we can now calculate the outputs from all attention heads with a single call to our self-attention operation.

In [24]:
all_heads_output, all_attn_weights = scaled_dot_product_attention(q_s_transposed, 
                                                                  k_s_transposed, 
                                                                  v_s_transposed)
print("Self attention output:\n", all_heads_output)

Self attention output:
 tf.Tensor(
[[[[2.8125799 3.0774028 4.2729797 3.0012922]
   [2.8190746 3.0672562 4.248918  3.004592 ]
   [2.8151314 3.0744383 4.265335  3.002575 ]]

  [[3.6123753 3.4325714 2.8910663 4.1445055]
   [3.5966477 3.426169  2.848069  4.1049523]
   [3.591525  3.435305  2.804224  4.0682306]]

  [[3.276435  2.7178073 3.2153256 3.2624912]
   [3.2268167 2.8030596 3.20815   3.2900314]
   [3.231248  2.79959   3.2109962 3.290989 ]]]], shape=(1, 3, 3, 4), dtype=float32)


As a sanity check, we can compare this against the outputs from individual heads we calculated earlier:

In [25]:
print("Per head outputs from using separate sets of weights per head:")
print(out0, "\n")
print(out1, "\n")
print(out2)

Per head outputs from using separate sets of weights per head:
tf.Tensor(
[[[2.8125799 3.0774028 4.2729797 3.0012922]
  [2.8190746 3.0672562 4.248918  3.004592 ]
  [2.8151314 3.0744383 4.265335  3.002575 ]]], shape=(1, 3, 4), dtype=float32) 

tf.Tensor(
[[[3.6123753 3.4325714 2.8910663 4.1445055]
  [3.5966477 3.426169  2.848069  4.1049523]
  [3.591525  3.435305  2.804224  4.0682306]]], shape=(1, 3, 4), dtype=float32) 

tf.Tensor(
[[[3.276435  2.7178073 3.2153256 3.2624912]
  [3.2268167 2.8030596 3.20815   3.2900314]
  [3.231248  2.79959   3.2109962 3.290989 ]]], shape=(1, 3, 4), dtype=float32)


To get the final concatenated result, we need to reverse our **reshape** and **transpose** operation, starting with the **transpose** this time.

In [26]:
combined_out_b = tf.reshape(tf.transpose(all_heads_output, perm=[0, 2, 1, 3]), 
                            shape=(batch_size, seq_len, embed_dim))
print("Final output from using single query, key, value matrices:\n", 
      combined_out_b, "\n")
print("Final output from using separate query, key, value matrices per head:\n", 
      combined_out_a)

Final output from using single query, key, value matrices:
 tf.Tensor(
[[[2.8125799 3.0774028 4.2729797 3.0012922 3.6123753 3.4325714 2.8910663
   4.1445055 3.276435  2.7178073 3.2153256 3.2624912]
  [2.8190746 3.0672562 4.248918  3.004592  3.5966477 3.426169  2.848069
   4.1049523 3.2268167 2.8030596 3.20815   3.2900314]
  [2.8151314 3.0744383 4.265335  3.002575  3.591525  3.435305  2.804224
   4.0682306 3.231248  2.79959   3.2109962 3.290989 ]]], shape=(1, 3, 12), dtype=float32) 

Final output from using separate query, key, value matrices per head:
 [[[2.8125799 3.0774028 4.2729797 3.0012922 3.6123753 3.4325714 2.8910663
   4.1445055 3.276435  2.7178073 3.2153256 3.2624912]
  [2.8190746 3.0672562 4.248918  3.004592  3.5966477 3.426169  2.848069
   4.1049523 3.2268167 2.8030596 3.20815   3.2900314]
  [2.8151314 3.0744383 4.265335  3.002575  3.591525  3.435305  2.804224
   4.0682306 3.231248  2.79959   3.2109962 3.290989 ]]]


We can encapsulate everything we just covered in a class.

In [27]:
class MultiHeadSelfAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadSelfAttention, self).__init__()
    self.d_model = d_model
    self.num_heads = num_heads

    self.d_head = self.d_model // self.num_heads

    self.wq = tf.keras.layers.Dense(self.d_model)
    self.wk = tf.keras.layers.Dense(self.d_model)
    self.wv = tf.keras.layers.Dense(self.d_model)

    # Linear layer to generate the final output.
    self.dense = tf.keras.layers.Dense(self.d_model)
  
  def split_heads(self, x):
    batch_size = x.shape[0]

    split_inputs = tf.reshape(x, (batch_size, -1, self.num_heads, self.d_head))
    return tf.transpose(split_inputs, perm=[0, 2, 1, 3])
  
  def merge_heads(self, x):
    batch_size = x.shape[0]

    merged_inputs = tf.transpose(x, perm=[0, 2, 1, 3])
    return tf.reshape(merged_inputs, (batch_size, -1, self.d_model))

  def call(self, q, k, v, mask):
    qs = self.wq(q)
    ks = self.wk(k)
    vs = self.wv(v)

    qs = self.split_heads(qs)
    ks = self.split_heads(ks)
    vs = self.split_heads(vs)

    output, attn_weights = scaled_dot_product_attention(qs, ks, vs, mask)
    output = self.merge_heads(output)

    return self.dense(output), attn_weights


In [28]:
mhsa = MultiHeadSelfAttention(12, 3)

output, attn_weights = mhsa(x, x, x, None)
print(f"MHSA output{output.shape}:")
print(output)

MHSA output(1, 3, 12):
tf.Tensor(
[[[ 0.60744333 -0.38894632 -0.13010728 -0.16148049 -0.11282776
    0.68603003 -0.81033516 -0.2734391  -0.29607397 -0.27549237
   -0.03618741  0.07487041]
  [ 0.6044789  -0.39247593 -0.16740322 -0.16347069 -0.1143138
    0.6770967  -0.78262925 -0.25929496 -0.2642433  -0.26896554
   -0.01960444  0.07027422]
  [ 0.5874938  -0.38159657 -0.15285817 -0.16249236 -0.135331
    0.67037666 -0.7676116  -0.26250648 -0.24591736 -0.25836378
   -0.03356725  0.06782886]]], shape=(1, 3, 12), dtype=float32)


## Encoder Block

We can now build our **Encoder Block**. In addition to the **Multi-Head Self Attention** layer, the **Encoder Block** also has **skip connections**, **layer normalization steps**, and a **two-layer feed-forward neural network**. The original **Attention Is All You Need** paper also included some **dropout** applied to the self-attention output which isn't shown in the illustration below (see references for a link to the paper).

<div>
<img src="https://drive.google.com/uc?export=view&id=1D8sLDyQMqqhCjHWOn-I7rZKHugWxFyLy" width="500"/>
</div>

Since a two-layer feed forward neural network is used in multiple places in the transformer, here's a function which creates and returns one.

In [29]:
def feed_forward_network(d_model, hidden_dim):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(hidden_dim, activation='relu'),
      tf.keras.layers.Dense(d_model)
  ])

This is our encoder block containing all the layers and steps from the preceding illustration (plus dropout).

In [30]:
class EncoderBlock(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, hidden_dim, dropout_rate=0.1):
    super(EncoderBlock, self).__init__()

    self.mhsa = MultiHeadSelfAttention(d_model, num_heads)
    self.ffn = feed_forward_network(d_model, hidden_dim)

    self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
    self.dropout2 = tf.keras.layers.Dropout(dropout_rate)

    self.layernorm1 = tf.keras.layers.LayerNormalization()
    self.layernorm2 = tf.keras.layers.LayerNormalization()
  
  def call(self, x, training, mask):
    mhsa_output, attn_weights = self.mhsa(x, x, x, mask)
    mhsa_output = self.dropout1(mhsa_output, training=training)
    mhsa_output = self.layernorm1(x + mhsa_output)

    ffn_output = self.ffn(mhsa_output)
    ffn_output = self.dropout2(ffn_output, training=training)
    output = self.layernorm2(mhsa_output + ffn_output)

    return output, attn_weights


Suppose we have an embedding dimension of 12, and we want 3 attention heads and a feed forward network with a hidden dimension of 48 (4x the embedding dimension). We would declare and use a single encoder block like so:

In [31]:
encoder_block = EncoderBlock(12, 3, 48)

block_output,  _ = encoder_block(x, True, None)
print(f"Output from single encoder block {block_output.shape}:")
print(block_output)

Output from single encoder block (1, 3, 12):
tf.Tensor(
[[[ 0.10154367 -0.91421366 -0.0794095   0.22988214 -0.12295797
    1.1260968   1.898228   -0.9752944   0.74560183 -0.02875669
   -2.1584797   0.1777595 ]
  [-0.40143302 -0.89792675  0.9674039  -0.66805047 -0.77481985
    1.0722231   1.4904646  -0.2931695   1.0017684  -1.4616723
   -1.1223446   1.0875565 ]
  [ 1.2802415  -1.6674963  -1.2316651  -0.82773596  0.04805525
    1.7087516   1.2304056  -0.08817066  0.16149795 -0.83812845
   -0.17961621  0.40386114]]], shape=(1, 3, 12), dtype=float32)


## Word and Positional Embeddings

Let's now deal with the actual input to the **initial** encoder block. The inputs are going to be *positional word embeddings*. That is, word embeddings with some positional information added to them.
<br>

Let's start with **subword** tokenization. For demonstration, we'll use a subword tokenizer called **BPEmb**. It uses **Byte-Pair Encoding** and supports over two hundred languages. 

https://bpemb.h-its.org/


In [32]:
# Load the English tokenizer.
bpemb_en = BPEmb(lang="en")

downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs10000.model


100%|██████████| 400869/400869 [00:00<00:00, 934977.58B/s]


downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs10000.d100.w2v.bin.tar.gz


100%|██████████| 3784656/3784656 [00:00<00:00, 4559127.59B/s]


The library comes with embeddings for a number of words.

In [33]:
bpemb_vocab_size, bpemb_embed_size = bpemb_en.vectors.shape
print("Vocabulary size:", bpemb_vocab_size)
print("Embedding size:", bpemb_embed_size)

Vocabulary size: 10000
Embedding size: 100


In [34]:
# Embedding for the word "car".
bpemb_en.vectors[bpemb_en.words.index('car')]

array([-0.305548, -0.325598, -0.134716, -0.078735, -0.660545,  0.076211,
       -0.735487,  0.124533, -0.294402,  0.459688,  0.030137,  0.174041,
       -0.224223,  0.486189, -0.504649, -0.459699,  0.315747,  0.477885,
        0.091398,  0.427867,  0.016524, -0.076833, -0.899727,  0.493158,
       -0.022309, -0.422785, -0.154148,  0.204981,  0.379834,  0.070588,
        0.196073, -0.368222,  0.473406,  0.007409,  0.004303, -0.007823,
       -0.19103 , -0.202509,  0.109878, -0.224521, -0.35741 , -0.611633,
        0.329958, -0.212956, -0.497499, -0.393839, -0.130101, -0.216903,
       -0.105595, -0.076007, -0.483942, -0.139704, -0.161647,  0.136985,
        0.415363, -0.360143,  0.038601, -0.078804, -0.030421,  0.324129,
        0.223378, -0.523636, -0.048317, -0.032248, -0.117367,  0.470519,
        0.225816, -0.222065, -0.225007, -0.165904, -0.334389, -0.20157 ,
        0.572352, -0.268794,  0.301929, -0.005563,  0.387491,  0.261031,
       -0.11613 ,  0.074982, -0.008433,  0.259987, 

We don't need the embeddings since we're going to use our own embedding layer. What we're interested in are the subword tokens and their respective ids. The ids will be used as indexes into our embedding layer.<br>

If this doesn't sound familiar, refer to the module on word vectors:<br>
https://www.nlpdemystified.org/course/word-vectors

These are the subword tokens for our example sentence from the slides. **BPEmb** places underscores in front of any tokens which are whole words or intended to begin words.<br>

Remember that subword tokenizers are trained using count frequencies over a corpus. So these subword tokens are specific to **BPEmb**. Another subword tokenizer may output something different. This is why it's important that when we use a pretrained model, we make sure to use the pretrained model's tokenizer. We'll see this when we use pretrained transformers later in this module.

In [35]:
sample_sentence = "Where can I find a pizzeria?"
tokens = bpemb_en.encode(sample_sentence)
print(tokens)

['▁where', '▁can', '▁i', '▁find', '▁a', '▁p', 'iz', 'zer', 'ia', '?']


We can retrieve each subword token's respective id using the *encode_ids* method.

In [36]:
token_seq = np.array(bpemb_en.encode_ids("Where can I find a pizzeria?"))
print(token_seq)

[ 571  280  386 1934    4   24  248 4339  177 9967]


Now that we have a way to tokenize and vectorize sentences, we can declare and use an embedding layer with the same vocabulary size as **BPEmb** and a desired embedding size.

In [37]:
token_embed = tf.keras.layers.Embedding(bpemb_vocab_size, embed_dim)
token_embeddings = token_embed(token_seq)

# The untrained embeddings for our sample sentence.
print("Embeddings for: ", sample_sentence)
print(token_embeddings)

Embeddings for:  Where can I find a pizzeria?
tf.Tensor(
[[ 0.00421689 -0.01556925  0.0120595  -0.03281087 -0.03540921  0.03880063
   0.0217952  -0.01293421  0.01441408 -0.0382198   0.01211702 -0.00013555]
 [-0.02838122  0.02070891  0.00359683  0.04353145  0.03331557 -0.00208879
   0.04910041 -0.02270471  0.01322749 -0.00966636 -0.01446652 -0.04066715]
 [ 0.00840092  0.03048452  0.01394225  0.00985941 -0.0405363   0.00406981
   0.01937666  0.01823227 -0.0049621   0.00180475 -0.01620211  0.0241042 ]
 [ 0.00630901 -0.02812046  0.02700213  0.02390398  0.01246969  0.01416733
  -0.02072176  0.04085294  0.02084127 -0.01128837  0.00127157 -0.04629595]
 [ 0.01738792  0.03850421  0.03149602 -0.01096026  0.02716782 -0.04935062
  -0.0095899  -0.04307562  0.02198527  0.0121994   0.03390704  0.00907252]
 [-0.03324686  0.01565409 -0.04642043 -0.00732569  0.00979467 -0.04835152
   0.0441843  -0.00566212 -0.02108947  0.01276419 -0.04907365 -0.03691318]
 [-0.01596306  0.04233887 -0.00360993 -0.03227114

Next, we need to add *positional* information to each token embedding. As we covered in the slides, the original paper used sinusoidals but it's more common these days to just use another set of embeddings. We'll do the latter here.<br>

Here, we're declaring an embedding layer with rows equalling a maximum sequence length and columns equalling our token embedding size. We then generate a vector of position ids.

In [38]:
max_seq_len = 256
pos_embed = tf.keras.layers.Embedding(max_seq_len, embed_dim)

# Generate ids for each position of the token sequence.
pos_idx = tf.range(len(token_seq))
print(pos_idx)

tf.Tensor([0 1 2 3 4 5 6 7 8 9], shape=(10,), dtype=int32)


We'll use these position ids to index into the positional embedding layer.

In [39]:
# These are our positon embeddings.
position_embeddings = pos_embed(pos_idx)
print("Position embeddings for the input sequence\n", position_embeddings)

Position embeddings for the input sequence
 tf.Tensor(
[[-0.01066743 -0.04011086 -0.00441873  0.01153155 -0.02596712  0.0080641
  -0.01685884 -0.01750354  0.04476124 -0.02939335 -0.02204077 -0.03786758]
 [-0.04832258 -0.00709655 -0.03279929 -0.03668386  0.04789474 -0.01683985
  -0.04325256 -0.01833801  0.02859024 -0.04088284  0.0401913   0.00142444]
 [ 0.02352054 -0.01071855  0.01703363  0.04530608 -0.04386398  0.00661146
   0.03708651  0.03134618  0.00867762  0.04625684  0.03111212  0.03935416]
 [-0.03096274 -0.03365568 -0.03280944 -0.04846409  0.01318463  0.00625678
  -0.00791824 -0.03157723 -0.02007229  0.01668389 -0.0250142  -0.04217333]
 [-0.00269855  0.03505993 -0.0193392  -0.04056039  0.02278418  0.0432458
  -0.0239082  -0.02554886 -0.00687215 -0.04253485 -0.03121564 -0.02144301]
 [-0.04556415 -0.00978464 -0.02093986  0.00805707  0.04898578  0.03055272
  -0.00045391 -0.0117264  -0.01840687  0.03050593 -0.03085827  0.0398556 ]
 [-0.01148916  0.04840806  0.01835584 -0.03943571  0.

The final step is to add our token and position embeddings. The result will be the input to the first encoder block.

In [40]:
input = token_embeddings + position_embeddings
print("Input to the initial encoder block:\n", input)

Input to the initial encoder block:
 tf.Tensor(
[[-0.00645055 -0.05568011  0.00764077 -0.02127931 -0.06137633  0.04686473
   0.00493636 -0.03043775  0.05917533 -0.06761315 -0.00992376 -0.03800314]
 [-0.0767038   0.01361236 -0.02920246  0.00684759  0.08121032 -0.01892864
   0.00584785 -0.04104272  0.04181772 -0.0505492   0.02572478 -0.03924271]
 [ 0.03192146  0.01976598  0.03097587  0.05516548 -0.08440027  0.01068126
   0.05646317  0.04957845  0.00371553  0.04806159  0.01491001  0.06345836]
 [-0.02465373 -0.06177614 -0.00580731 -0.02456011  0.02565432  0.02042412
  -0.02864     0.00927571  0.00076897  0.00539552 -0.02374263 -0.08846928]
 [ 0.01468936  0.07356414  0.01215682 -0.05152065  0.049952   -0.00610482
  -0.0334981  -0.06862448  0.01511311 -0.03033545  0.0026914  -0.01237049]
 [-0.07881102  0.00586945 -0.0673603   0.00073138  0.05878046 -0.0177988
   0.04373039 -0.01738852 -0.03949634  0.04327012 -0.07993191  0.00294242]
 [-0.02745222  0.09074693  0.0147459  -0.07170685  0.062127

## Encoder

Now that we have an encoder block and a way to embed our tokens with position information, we can create the **encoder** itself.<br>

Given a batch of vectorized sequences, the encoder creates positional embeddings, runs them through its encoder blocks, and returns contextualized tokens.

In [41]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_blocks, d_model, num_heads, hidden_dim, src_vocab_size,
               max_seq_len, dropout_rate=0.1):
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.max_seq_len = max_seq_len

    self.token_embed = tf.keras.layers.Embedding(src_vocab_size, self.d_model)
    self.pos_embed = tf.keras.layers.Embedding(max_seq_len, self.d_model)

    # The original Attention Is All You Need paper applied dropout to the
    # input before feeding it to the first encoder block.
    self.dropout = tf.keras.layers.Dropout(dropout_rate)

    # Create encoder blocks.
    self.blocks = [EncoderBlock(self.d_model, num_heads, hidden_dim, dropout_rate) 
    for _ in range(num_blocks)]
  
  def call(self, input, training, mask):
    token_embeds = self.token_embed(input)

    # Generate position indices for a batch of input sequences.
    num_pos = input.shape[0] * self.max_seq_len
    pos_idx = np.resize(np.arange(self.max_seq_len), num_pos)
    pos_idx = np.reshape(pos_idx, input.shape)
    pos_embeds = self.pos_embed(pos_idx)

    x = self.dropout(token_embeds + pos_embeds, training=training)

    # Run input through successive encoder blocks.
    for block in self.blocks:
      x, weights = block(x, training, mask)

    return x, weights

If you're wondering about this code block here:


```
num_pos = input.shape[0] * self.max_seq_len
pos_idx = np.resize(np.arange(self.max_seq_len), num_pos)
pos_idx = np.reshape(pos_idx, input.shape)
pos_embeds = self.pos_embed(pos_idx)
```


This generates positional embeddings for a *batch* of input sequences. Suppose this was our batch of input sequences to the encoder.

In [42]:
# Batch of 3 sequences, each of length 10 (10 is also the 
# maximum sequence length in this case).
seqs = np.random.randint(0, 10000, size=(3, 10))
print(seqs.shape)
print(seqs)

(3, 10)
[[1631 7822 9580 3215 6193 7367 1507 3810 5339 5384]
 [6058 6089 5681 7826 7854 6658 9342 9032 9564 6319]
 [5719 4735 2673 3631 6410 3601 8138 8080 7863 2911]]


We need to retrieve a positional embedding for every element in this batch. The first step is to create the respective positional ids...

In [43]:
pos_ids = np.resize(np.arange(seqs.shape[1]), seqs.shape[0] * seqs.shape[1])
print(pos_ids)

[0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9]


...and then reshape them to match the input batch dimensions.

In [44]:
pos_ids = np.reshape(pos_ids, (3, 10))
print(pos_ids.shape)
print(pos_ids)

(3, 10)
[[0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]]


We can now retrieve position embeddings for every token embedding.

In [45]:
pos_embed(pos_ids)

<tf.Tensor: shape=(3, 10, 12), dtype=float32, numpy=
array([[[-0.01066743, -0.04011086, -0.00441873,  0.01153155,
         -0.02596712,  0.0080641 , -0.01685884, -0.01750354,
          0.04476124, -0.02939335, -0.02204077, -0.03786758],
        [-0.04832258, -0.00709655, -0.03279929, -0.03668386,
          0.04789474, -0.01683985, -0.04325256, -0.01833801,
          0.02859024, -0.04088284,  0.0401913 ,  0.00142444],
        [ 0.02352054, -0.01071855,  0.01703363,  0.04530608,
         -0.04386398,  0.00661146,  0.03708651,  0.03134618,
          0.00867762,  0.04625684,  0.03111212,  0.03935416],
        [-0.03096274, -0.03365568, -0.03280944, -0.04846409,
          0.01318463,  0.00625678, -0.00791824, -0.03157723,
         -0.02007229,  0.01668389, -0.0250142 , -0.04217333],
        [-0.00269855,  0.03505993, -0.0193392 , -0.04056039,
          0.02278418,  0.0432458 , -0.0239082 , -0.02554886,
         -0.00687215, -0.04253485, -0.03121564, -0.02144301],
        [-0.04556415, -0.00

Let's try our encoder on a batch of sentences.

In [46]:
input_batch = [
    "Where can I find a pizzeria?",
    "Mass hysteria over listeria.",
    "I ain't no circle back girl."
]

bpemb_en.encode(input_batch)

[['▁where', '▁can', '▁i', '▁find', '▁a', '▁p', 'iz', 'zer', 'ia', '?'],
 ['▁mass', '▁hy', 'ster', 'ia', '▁over', '▁l', 'ister', 'ia', '.'],
 ['▁i', '▁a', 'in', "'", 't', '▁no', '▁circle', '▁back', '▁girl', '.']]

In [47]:
input_seqs = bpemb_en.encode_ids(input_batch)
print("Vectorized inputs:")
input_seqs

Vectorized inputs:


[[571, 280, 386, 1934, 4, 24, 248, 4339, 177, 9967],
 [1535, 1354, 1238, 177, 380, 43, 871, 177, 9935],
 [386, 4, 6, 9937, 9915, 467, 5410, 810, 3692, 9935]]

Note how the input sequences aren't the same length in this batch. In this case, we need to pad them out so that they are. If you're unfamiliar with why, refer to the notebook on Recurrent Neural Networks:<br>
https://colab.research.google.com/github/nitinpunjabi/nlp-demystified/blob/main/notebooks/nlpdemystified_recurrent_neural_networks.ipynb<br>

We'll do this using *pad_sequences*.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/utils/pad_sequences

In [48]:
padded_input_seqs = tf.keras.preprocessing.sequence.pad_sequences(input_seqs, padding="post")
print("Input to the encoder:")
print(padded_input_seqs.shape)
print(padded_input_seqs)

Input to the encoder:
(3, 10)
[[ 571  280  386 1934    4   24  248 4339  177 9967]
 [1535 1354 1238  177  380   43  871  177 9935    0]
 [ 386    4    6 9937 9915  467 5410  810 3692 9935]]


Since our input now has padding, now's a good time to cover **masking**.
<br>

So given a mask, wherever there's a mask position set to 0, the corresponding position in the attention scores will be set to *-inf*. The resulting attention weight for the position will then be zero and no attending will occur for that position.
<br>

In the slides, we covered *look-ahead* masks for the decoder to prevent it from attending to future tokens, but we also need masks for padding.
<br>

In total, there are three masks involved:
1. The *encoder mask* to mask out any padding in the encoder sequences.

2. The *decoder mask* which is used in the decoder's **first** multi-head self-attention layer. It's a <u>combination of two masks</u>: one to account for the padding in target sequences, and the look-ahead mask.

3. The *memory mask* which is used in the decoder's **second** multi-head self-attention layer. The keys and values for this layer are going to be the encoder's output, and this mask will ensure the decoder doesn't attend to any encoder output which corresponds to padding. In practice, 1 and 3 are often the same.

The *scaled_dot_product_attention* function has this line:
```
  if mask is not None:
    scaled_scores = tf.where(mask==0, -np.inf, scaled_scores)
```

Let's create an encoder mask for our batch of input sequences.<br>

Wherever there's padding, we want the mask position set to zero.

In [49]:
enc_mask = tf.cast(tf.math.not_equal(padded_input_seqs, 0), tf.float32)
print("Input:")
print(padded_input_seqs, '\n')
print("Encoder mask:")
print(enc_mask)

Input:
[[ 571  280  386 1934    4   24  248 4339  177 9967]
 [1535 1354 1238  177  380   43  871  177 9935    0]
 [ 386    4    6 9937 9915  467 5410  810 3692 9935]] 

Encoder mask:
tf.Tensor(
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]], shape=(3, 10), dtype=float32)


Keep in mind that the dimension of the attention matrix (for this example) is going to be:<br>
*(batch size, number of heads, query size, key size)*<br>
(3, 3, 10, 10)

So we need to expand the mask dimensions like so:

In [50]:
enc_mask = enc_mask[:, tf.newaxis, tf.newaxis, :]
enc_mask

<tf.Tensor: shape=(3, 1, 1, 10), dtype=float32, numpy=
array([[[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]],


       [[[1., 1., 1., 1., 1., 1., 1., 1., 1., 0.]]],


       [[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]]], dtype=float32)>

This way, the encoder mask will now be *broadcasted*.<br>
https://www.tensorflow.org/xla/broadcasting

Now we can declare an encoder and pass it batches of vectorized sequences.

In [51]:
num_encoder_blocks = 6

# d_model is the embedding dimension used throughout.
d_model = 12

num_heads = 3

# Feed-forward network hidden dimension width.
ffn_hidden_dim = 48

src_vocab_size = bpemb_vocab_size
max_input_seq_len = padded_input_seqs.shape[1]

encoder = Encoder(
    num_encoder_blocks,
    d_model,
    num_heads,
    ffn_hidden_dim,
    src_vocab_size,
    max_input_seq_len)

We can now pass our input sequences and mask to the encoder.

In [52]:
encoder_output, attn_weights = encoder(padded_input_seqs, training=True, 
                                       mask=enc_mask)
print(f"Encoder output {encoder_output.shape}:")
print(encoder_output)

Encoder output (3, 10, 12):
tf.Tensor(
[[[-1.7272385  -1.3047042   0.08937338 -0.78903973  0.5774019
    1.5966614   0.2508738  -0.07100187  0.73957366  1.6313947
   -0.38684303 -0.6064516 ]
  [-1.2064242  -1.1460967   0.53783363 -1.0603099   0.7454572
    1.6689852  -1.1059935   0.16496652  1.3632755   0.82482153
   -0.8243567   0.03784112]
  [-0.6890051  -1.1856745   0.9993422  -1.3975056   1.0245456
    0.58452857 -0.63539594 -0.0366108   2.134816    0.26609933
   -0.82143134 -0.24370806]
  [-1.3357496  -1.5653051   1.1235867  -0.3353301   1.06686
    0.8735445  -1.1880167   1.2905284   1.019442   -0.16507168
   -0.26764643 -0.51684195]
  [-1.2707527  -1.1013126   1.2773265  -1.0646877   1.1125014
    0.7191851  -0.8532318   0.18328226  1.6670867  -0.34071636
   -0.8471317   0.5184512 ]
  [-1.8106623  -1.4625796   0.224943   -0.485935    0.8530487
    1.0428362  -0.6537574  -0.09332516  1.3818741   1.4007797
    0.01023787 -0.40746003]
  [-2.029312   -0.4979887   0.560798   -1.23523

## Decoder Block

Let's build the **Decoder Block**. Everything we did to create the **encoder** block applies here. The major differences are that the **Decoder Block** has:
1. a **Multi-Head Cross-Attention** layer which uses the encoder's outputs as the keys and values.

2. an extra skip/residual connection along with an extra layer normalization step.

<div>
<img src="https://drive.google.com/uc?export=view&id=1WVT4SX49bnta4uscOTF4xrsxFI4PbPER" width="500"/>
</div>

In [53]:
class DecoderBlock(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, hidden_dim, dropout_rate=0.1):
    super(DecoderBlock, self).__init__()

    self.mhsa1 = MultiHeadSelfAttention(d_model, num_heads)
    self.mhsa2 = MultiHeadSelfAttention(d_model, num_heads)

    self.ffn = feed_forward_network(d_model, hidden_dim)

    self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
    self.dropout2 = tf.keras.layers.Dropout(dropout_rate)
    self.dropout3 = tf.keras.layers.Dropout(dropout_rate)

    self.layernorm1 = tf.keras.layers.LayerNormalization()
    self.layernorm2 = tf.keras.layers.LayerNormalization()
    self.layernorm3 = tf.keras.layers.LayerNormalization()
  
  # Note the decoder block takes two masks. One for the first MHSA, another
  # for the second MHSA.
  def call(self, encoder_output, target, training, decoder_mask, memory_mask):
    mhsa_output1, attn_weights = self.mhsa1(target, target, target, decoder_mask)
    mhsa_output1 = self.dropout1(mhsa_output1, training=training)
    mhsa_output1 = self.layernorm1(mhsa_output1 + target)

    mhsa_output2, attn_weights = self.mhsa2(mhsa_output1, encoder_output, 
                                            encoder_output, 
                                            memory_mask)
    mhsa_output2 = self.dropout2(mhsa_output2, training=training)
    mhsa_output2 = self.layernorm2(mhsa_output2 + mhsa_output1)

    ffn_output = self.ffn(mhsa_output2)
    ffn_output = self.dropout3(ffn_output, training=training)
    output = self.layernorm3(ffn_output + mhsa_output2)

    return output, attn_weights


## Decoder

The decoder is almost the same as the encoder except it takes the encoder's output as part of its input, and it takes two masks: the decoder mask and memory mask.

In [54]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, num_blocks, d_model, num_heads, hidden_dim, target_vocab_size,
               max_seq_len, dropout_rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.max_seq_len = max_seq_len

    self.token_embed = tf.keras.layers.Embedding(target_vocab_size, self.d_model)
    self.pos_embed = tf.keras.layers.Embedding(max_seq_len, self.d_model)

    self.dropout = tf.keras.layers.Dropout(dropout_rate)

    self.blocks = [DecoderBlock(self.d_model, num_heads, hidden_dim, dropout_rate) for _ in range(num_blocks)]

  def call(self, encoder_output, target, training, decoder_mask, memory_mask):
    token_embeds = self.token_embed(target)

    # Generate position indices.
    num_pos = target.shape[0] * self.max_seq_len
    pos_idx = np.resize(np.arange(self.max_seq_len), num_pos)
    pos_idx = np.reshape(pos_idx, target.shape)

    pos_embeds = self.pos_embed(pos_idx)

    x = self.dropout(token_embeds + pos_embeds, training=training)

    for block in self.blocks:
      x, weights = block(encoder_output, x, training, decoder_mask, memory_mask)

    return x, weights

Before we try the decoder, let's cover the masks involved. The decoder takes two masks:

The *decoder mask* which is a <u>combination of two masks</u>: one to account for the padding in target sequences, and the look-ahead mask. This mask is used in the decoder's **first** multi-head self-attention layer.

The *memory mask* which is used in the decoder's **second** multi-head self-attention. The keys and values for this layer are going to be the encoder's output, and this mask will ensure the decoder doesn't attend to any encoder output which corresponds to padding.

Suppose this is our batch of vectorized target *input* sequences for the decoder. These values are just made up.<br>

**Note**: If you need a refresher on how to prepare target input and output sequences for the decoder, refer to the [seq2seq notebook](https://colab.research.google.com/github/nitinpunjabi/nlp-demystified/blob/main/notebooks/nlpdemystified_seq2seq_and_attention.ipynb).



In [55]:
# Made up values.
target_input_seqs = [
    [1, 652, 723, 123, 62],
    [1, 25,  98, 129, 248, 215, 359, 249],
    [1, 2369, 1259, 125, 486],
]

As we did with the encoder input sequences, we need to pad out this batch so that all sequences within it are the same length.

In [56]:
padded_target_input_seqs = tf.keras.preprocessing.sequence.pad_sequences(target_input_seqs, padding="post")
print("Padded target inputs to the decoder:")
print(padded_target_input_seqs.shape)
print(padded_target_input_seqs)

Padded target inputs to the decoder:
(3, 8)
[[   1  652  723  123   62    0    0    0]
 [   1   25   98  129  248  215  359  249]
 [   1 2369 1259  125  486    0    0    0]]


We can create the padding mask the same way we did for the encoder.

In [57]:
dec_padding_mask = tf.cast(tf.math.not_equal(padded_target_input_seqs, 0), tf.float32)
dec_padding_mask = dec_padding_mask[:, tf.newaxis, tf.newaxis, :]
print(dec_padding_mask)

tf.Tensor(
[[[[1. 1. 1. 1. 1. 0. 0. 0.]]]


 [[[1. 1. 1. 1. 1. 1. 1. 1.]]]


 [[[1. 1. 1. 1. 1. 0. 0. 0.]]]], shape=(3, 1, 1, 8), dtype=float32)


As we covered in the slides, the look-ahead mask is a diagonal where the lower half are 1s and the upper half are zeros. This is easy to create using the *band_part* method:<br>
https://www.tensorflow.org/api_docs/python/tf/linalg/band_part

In [58]:
target_input_seq_len = padded_target_input_seqs.shape[1]
look_ahead_mask = tf.linalg.band_part(tf.ones((target_input_seq_len, 
                                               target_input_seq_len)), -1, 0)
print(look_ahead_mask)

tf.Tensor(
[[1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1.]], shape=(8, 8), dtype=float32)


To create the decoder mask, we just need to combine the padding and look-ahead masks. Note how the columns of the resulting decoder mask are all zero for padding positions.

In [59]:
dec_mask = tf.minimum(dec_padding_mask, look_ahead_mask)
print("The decoder mask:")
print(dec_mask)

The decoder mask:
tf.Tensor(
[[[[1. 0. 0. 0. 0. 0. 0. 0.]
   [1. 1. 0. 0. 0. 0. 0. 0.]
   [1. 1. 1. 0. 0. 0. 0. 0.]
   [1. 1. 1. 1. 0. 0. 0. 0.]
   [1. 1. 1. 1. 1. 0. 0. 0.]
   [1. 1. 1. 1. 1. 0. 0. 0.]
   [1. 1. 1. 1. 1. 0. 0. 0.]
   [1. 1. 1. 1. 1. 0. 0. 0.]]]


 [[[1. 0. 0. 0. 0. 0. 0. 0.]
   [1. 1. 0. 0. 0. 0. 0. 0.]
   [1. 1. 1. 0. 0. 0. 0. 0.]
   [1. 1. 1. 1. 0. 0. 0. 0.]
   [1. 1. 1. 1. 1. 0. 0. 0.]
   [1. 1. 1. 1. 1. 1. 0. 0.]
   [1. 1. 1. 1. 1. 1. 1. 0.]
   [1. 1. 1. 1. 1. 1. 1. 1.]]]


 [[[1. 0. 0. 0. 0. 0. 0. 0.]
   [1. 1. 0. 0. 0. 0. 0. 0.]
   [1. 1. 1. 0. 0. 0. 0. 0.]
   [1. 1. 1. 1. 0. 0. 0. 0.]
   [1. 1. 1. 1. 1. 0. 0. 0.]
   [1. 1. 1. 1. 1. 0. 0. 0.]
   [1. 1. 1. 1. 1. 0. 0. 0.]
   [1. 1. 1. 1. 1. 0. 0. 0.]]]], shape=(3, 1, 8, 8), dtype=float32)


We can now declare a decoder and pass it everything it needs. In our case, the *memory* mask is the same as the *encoder* mask.

In [60]:
decoder = Decoder(6, 12, 3, 48, 10000, 8)
decoder_output, _ = decoder(encoder_output, padded_target_input_seqs, 
                            True, dec_mask, enc_mask)
print(f"Decoder output {decoder_output.shape}:")
print(decoder_output)

Decoder output (3, 8, 12):
tf.Tensor(
[[[-5.81482530e-01  1.44390273e+00 -1.35344565e+00 -6.86533898e-02
    2.01029792e-01 -5.99461317e-01 -4.04486097e-02  1.75699365e+00
   -1.58179665e+00 -5.01203775e-01  1.21574950e+00  1.08816378e-01]
  [ 7.59538710e-02  1.36686194e+00 -1.61201620e+00  6.48330510e-01
    6.31701708e-01 -1.40309775e+00  3.89280945e-01  1.47070980e+00
   -1.03901422e+00 -8.18876624e-01  7.03953266e-01 -4.13787156e-01]
  [-3.19918096e-01  1.60993540e+00 -1.03605390e+00  7.40746617e-01
    4.16802347e-01 -1.22831416e+00  4.96237367e-01  1.51110923e+00
   -1.12468147e+00 -1.21367919e+00  6.71476722e-01 -5.23660719e-01]
  [ 2.81474471e-01  1.49883091e+00 -1.21337163e+00 -1.18997298e-01
    7.57166505e-01 -1.31888282e+00  4.65257347e-01  1.47129440e+00
   -1.10230243e+00 -1.23088670e+00  8.13941479e-01 -3.03524345e-01]
  [-1.51320651e-01  1.70103407e+00 -1.26098990e+00  4.79107797e-01
    6.03541672e-01 -8.20733547e-01  1.57709733e-01  1.48971999e+00
   -9.95200515e-01 -

## Transformer

We now have all the pieces to build the **Transformer** itself, and it's pretty simple. 

In [61]:
class Transformer(tf.keras.Model):
  def __init__(self, num_blocks, d_model, num_heads, hidden_dim, source_vocab_size,
               target_vocab_size, max_input_len, max_target_len, dropout_rate=0.1):
    super(Transformer, self).__init__()

    self.encoder = Encoder(num_blocks, d_model, num_heads, hidden_dim, source_vocab_size, 
                           max_input_len, dropout_rate)
    
    self.decoder = Decoder(num_blocks, d_model, num_heads, hidden_dim, target_vocab_size,
                           max_target_len, dropout_rate)
    
    # The final dense layer to generate logits from the decoder output.
    self.output_layer = tf.keras.layers.Dense(target_vocab_size)

  def call(self, input_seqs, target_input_seqs, training, encoder_mask,
           decoder_mask, memory_mask):
    encoder_output, encoder_attn_weights = self.encoder(input_seqs, 
                                                        training, encoder_mask)

    decoder_output, decoder_attn_weights = self.decoder(encoder_output, 
                                                        target_input_seqs, training,
                                                        decoder_mask, memory_mask)

    return self.output_layer(decoder_output), encoder_attn_weights, decoder_attn_weights


In [62]:
transformer = Transformer(
    num_blocks = 6,
    d_model = 12,
    num_heads = 3,
    hidden_dim = 48,
    source_vocab_size = bpemb_vocab_size,
    target_vocab_size = 7000, # made-up target vocab size.
    max_input_len = padded_input_seqs.shape[1],
    max_target_len = padded_target_input_seqs.shape[1])

transformer_output, _, _ = transformer(padded_input_seqs, 
                                       padded_target_input_seqs, True, 
                                       enc_mask, dec_mask, memory_mask=enc_mask)
print(f"Transformer output {transformer_output.shape}:")
print(transformer_output) # If training, we would use this output to calculate losses.

Transformer output (3, 8, 7000):
tf.Tensor(
[[[-0.04351604 -0.0287575  -0.03005916 ...  0.01556304 -0.04024496
    0.05711899]
  [-0.04107515 -0.01682479 -0.02604807 ...  0.02714645 -0.06445252
    0.04676599]
  [-0.04542756  0.00229171 -0.02464298 ... -0.02922224 -0.05412746
    0.04309491]
  ...
  [-0.05639559 -0.01634248 -0.00367495 ... -0.01865666 -0.06173295
    0.05739674]
  [-0.04812533 -0.00453325 -0.00935392 ... -0.03824693  0.00993992
    0.03595619]
  [-0.03537487 -0.02338742 -0.06736737 ... -0.02150837 -0.02967767
    0.05378253]]

 [[-0.00695507 -0.00437397  0.01788752 ... -0.07259025 -0.01625334
    0.04398295]
  [-0.03028662 -0.01618615  0.02189036 ... -0.07033243 -0.03479235
    0.03994537]
  [-0.02598205 -0.03964914 -0.02140875 ... -0.05443469 -0.0142064
    0.06435058]
  ...
  [ 0.01420836  0.00920238  0.00897237 ... -0.06751305 -0.02032485
    0.0506382 ]
  [ 0.03377833 -0.04122647 -0.02833087 ... -0.08382164 -0.01289345
    0.07922615]
  [ 0.02613192  0.01111025  0.

That's the whole original transformer from scratch. From here, if you want to train this transformer, you can use the same approach we used when we built the translation model with attention in the [seq2seq notebook](https://colab.research.google.com/github/nitinpunjabi/nlp-demystified/blob/main/notebooks/nlpdemystified_seq2seq_and_attention.ipynb#scrollTo=x8Ef_eWXjWMn&line=3&uniqifier=1). Remember to use a learning rate warmup (Refer to the paper for more information on this).

It's useful to know how these models work under the hood, but to train our own transformer to get impressive results is expensive. Both in terms of compute and data.<br>

Fortunately, there's a zoo of **pretrained** transformer models we can use. We'll explore that next.

# Pre-Training and Transfer Learning with Hugging Face and OpenAI

**IMPORTANT**<br>
Enable **GPU acceleration** by going to *Runtime > Change Runtime Type*. Keep in mind that, on certain tiers, you're not guaranteed GPU access depending on usage history and current load.
<br><br>
Also, if you're running this in the cloud rather than a local Jupyter server on your machine, then the notebook will *timeout* after a period of inactivity.
<br><br>
Refer to this link on how to run Colab notebooks locally on your machine to avoid this issue:<br>
https://research.google.com/colaboratory/local-runtimes.html

We'll explore pre-training and transfer learning using the **Transformers** library from [Hugging Face](https://huggingface.co/). **Transformers** is an API and toolkit to download pre-trained models and further train them as needed. <br>

We'll start with the **pipelines** module which abstracts a lot of operations such as tokenization, vectorization, inference, etc.<br>

With **Transformers pipelines**, we can just feed text input and get text output. And there are **pipelines** for common tasks including classification, NER, summarization, etc.<br>
https://huggingface.co/docs/transformers/index<br>
https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#pipelines

To get started, we'll need to install **Transformers**.

In [63]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 93.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 22.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 11.0 MB/s eta 0:00:0

In [64]:
import operator
import pandas as pd
import tensorflow as tf
import transformers

from datasets import load_dataset
from tensorflow import keras
from transformers import AutoTokenizer
from transformers import pipeline
from transformers import TFAutoModelForQuestionAnswering

## Getting up and running quickly with Hugging Face Pipelines

We'll use the **pipeline** (note the singular) abstraction which wraps all the other pipelines. Put simply, it'll be our interface to doing a bunch of NLP tasks.

Using the **pipeline** abstraction is easy. We can instantiate a pipeline with a particular task, and it'll automatically download a suitable tokenizer and model behind the scenes for us and take care of the input and output operations.<br>
https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline<br>



Here, we're retrieving a pipeline for text-classification.

In [65]:
classifier = pipeline("text-classification")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Note the warning message about how no model was supplied. When we instantiate a pipeline for a task without specifying a particular model to perform the task, **Transformers** uses a default model. This is good enough for prototyping but for production, we'll want to specify which model to use for the task since the default can change. We'll see how to do this further below.

We can use the pipeline immediately to classify some text. Tokenization, vectorization, etc is taken care of behind the scenes.

In [66]:
classifier("Alice was excited to go the island but it didn't live up to the hype.")

[{'label': 'NEGATIVE', 'score': 0.9993934631347656}]

In [67]:
classifier("Bob doesn't do well in group situations but he said it wasn't bad.")

[{'label': 'POSITIVE', 'score': 0.9946909546852112}]

There's support for summarization...

In [68]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [69]:
text = """
Hans Niemann is launching a counterattack in his dispute with chess world 
champion Magnus Carlsen, filing a federal lawsuit that accuses Carlsen of 
maliciously colluding with others to defame the 19-year-old grandmaster and 
ruin his career.

It's the latest move in a scandal that has injected unprecedented levels of 
drama into the world of elite chess since early September, when Carlsen 
suggested Niemann's upset victory over him at the Sinquefield Cup tournament 
in St. Louis was the result of cheating.

Niemann wants a federal court in Missouri's eastern district to award him at 
least $100 million in damages. Defendants in the lawsuit include Carlsen, his 
company Play Magnus Group, the online platform Chess.com and its leader, Danny 
Rensch, along with grandmaster Hikaru Nakamura.
"""

In [70]:
summarizer(text)

[{'summary_text': ' Chess grandmaster Hans Niemann files federal lawsuit against Magnus Carlsen . He accuses Carlsen of colluding with others to defame the 19-year-old grandmaster . Defendants in the lawsuit include Carlsen, Play Magnus Group, the online platform Chess.com and its leader, Danny Rensch .'}]

...and question answering (extractive in this example).

In [71]:
qa = pipeline("question-answering")

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [72]:
context="""
Hugging Face was founded in 2016 by Clément Delangue, Julien Chaumond, and 
Thomas Wolf originally as a company that developed a chatbot app targeted at 
teenagers.[2] After open-sourcing the model behind the chatbot, the company 
pivoted to focus on being a platform for democratizing machine learning. In March 
2021, Hugging Face raised $40 million in a Series B funding round.
"""

question = "Who are the Hugging Face founders?"

qa(question=question, context=context)

{'score': 0.9919217228889465,
 'start': 37,
 'end': 88,
 'answer': 'Clément Delangue, Julien Chaumond, and \nThomas Wolf'}

Extractive question-answering models work fine for certain domains, document structures, and questions. But situations that require reasoning, more complex parsing, or contain ambiguity can trip it up.

In [73]:
question = "What does Hugging Face do?"
qa(question=question, context=context)

{'score': 0.08730360865592957,
 'start': 118,
 'end': 164,
 'answer': 'developed a chatbot app targeted at \nteenagers'}

There are ready-made pipelines for a number of tasks:<br>
https://huggingface.co/docs/transformers/main/en/quicktour#pipeline

Let's say we want a pipeline that uses a particular model. On the Hugging Face model hub, you'll find both pre-trained models (e.g. BERT) *and* pre-trained models that have been fine-tuned for all sorts of tasks (e.g. BERT for text classification). These models are contributed by Hugging Face, other companies, institutions, and individuals. You can (and are encouraged) to train or fine-tune a model and upload it for others to use.<br>
https://huggingface.co/models
<br><br>
For example, here's a collection of pre-trained models that have been tuned for text classification.<br>
https://huggingface.co/models?pipeline_tag=text-classification&sort=downloads
<br><br>
This particular one is a pre-trained *Roberta-base* model that's been fine-tuned on Twitter data for sentiment analysis:
https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment
<br><br>
Note how you can try the model directly on the model page.

Let's say we want to download and use a particular model. For example, this *BERT-base* model fine-tuned for NER:
https://huggingface.co/dslim/bert-base-NER
<br>

We just need to pass the model path during pipeline instantiation.

In [74]:
ner = pipeline(model="dslim/bert-base-NER")

In [75]:
text = "Panic ensues in Redmond as love child of Microsoft and OpenAI declares humanity obsolete." 
ner(text)

[{'entity': 'B-PER',
  'score': 0.9993875,
  'index': 6,
  'word': 'Red',
  'start': 16,
  'end': 19},
 {'entity': 'I-PER',
  'score': 0.8049695,
  'index': 7,
  'word': '##mond',
  'start': 19,
  'end': 23},
 {'entity': 'B-ORG',
  'score': 0.9980654,
  'index': 12,
  'word': 'Microsoft',
  'start': 41,
  'end': 50},
 {'entity': 'B-ORG',
  'score': 0.9985505,
  'index': 14,
  'word': 'Open',
  'start': 55,
  'end': 59},
 {'entity': 'I-ORG',
  'score': 0.98842865,
  'index': 15,
  'word': '##A',
  'start': 59,
  'end': 60},
 {'entity': 'I-ORG',
  'score': 0.9739822,
  'index': 16,
  'word': '##I',
  'start': 60,
  'end': 61}]

The **Transformers** library provides a bunch of helper classes to help with training models. And beyond the model hub, Hugging Face also hosts datasets, provides *spaces* where you can host your app, and offers a bunch of services such as cloud hardware and inference endpoints to help deploy your model.<br>
Datasets: https://huggingface.co/datasets<br>
Spaces: https://huggingface.co/spaces<br>

With Hugging Face, you can build an ML app prototype within minutes and iterate quickly from there.<br>
https://huggingface.co/docs<br>

Learn more about how to build with Hugging Face through their free course and fantastic book:<br>
https://huggingface.co/course<br>
https://www.oreilly.com/library/view/natural-language-processing/9781098136789/


## Fine-Tuning a Pre-Trained Model.

Let's say the model hub doesn't have a model that exactly suits your purpose. Perhaps you work in a particular domain and need to fine-tune a model using your own dataset.<br>

In this section, we'll walk through how to download a pre-trained model and fine-tune it. Our example covers extractive question answering but it's the same idea with other tasks.

We'll fine-tune using a dataset from the **Datasets** hub.<br>
https://huggingface.co/datasets<br><br>
Hugging Face provides a **datasets** library to download and interact with the datasets. It's similar to the Tensorflow Dataset library we used in that it can hold data and provides a bunch of methods to preprocess that data.<br>
https://huggingface.co/docs/datasets/ndex

The **Datasets** hub holds a bunch of question answering datasets.<br>
https://huggingface.co/datasets?task_categories=task_categories:question-answering&sort=downloads<br>


They differ based on data source, domain, and level of challenge. Since we're in a constrained environment (Colab free tier) and just learning how to fine-tune, we'll use SQuAD, a famous dataset comprised of crowd-sourced questions on a set of Wikipedia articles, and where the answer is a span of text in the article.<br>
https://huggingface.co/datasets/squad


In [76]:
data = load_dataset("squad")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

The **datasets** library downloads and automatically splits the data into train and validation sets. It returns a dictionary of **Dataset** objects:<br>
https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.DatasetDict<br>
https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset<br>


A **Dataset** object wraps an Apache Arrow table and provides a bunch of helper functions on top of it.<br>
https://arrow.apache.org/

In [77]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

Glaning at the data, we see every context (Wikipedia passage) is used multiple times. i.e., there are multiple questions and answers for each context.<br>

Every answer is a span of text from the context and the character position where the answer starts in the context is given.

In [78]:
pd.DataFrame(data['train'][0, 1, 2, 100, 101, 102], 
             columns=["context", "question", "answers"])

,context,question,answers
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{'text': ['Saint Bernadette Soubirous'], 'answ..."
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,"{'text': ['a copper statue of Christ'], 'answe..."
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,"{'text': ['the Main Building'], 'answer_start'..."
3,One of the main driving forces in the growth o...,In what year did the team lead by Knute Rockne...,"{'text': ['1925'], 'answer_start': [354]}"
4,One of the main driving forces in the growth o...,How many years was Knute Rockne head coach at ...,"{'text': ['13'], 'answer_start': [251]}"
5,One of the main driving forces in the growth o...,How many national titles were won when Knute R...,"{'text': ['three'], 'answer_start': [274]}"


Here's what we need to do:
1. Choose a pre-trained model based on what we want to accomplish and our constraints.
2. Download the appropriate tokenizer for the pre-trained model.
3. Tokenize and vectorize our dataset.
4. Mark where each answer starts and ends in our vectorized dataset.
5. Download the pre-trained model.
6. Fine-tune the pre-trained model with the vectorized dataset.

Given the free tier of Colab doesn't have a lot of GPU memory and that we're just trying to fine-tune a simple, extractive question answering model, we'll use *distilroberta-base*.<br>
https://huggingface.co/distilroberta-base
<br><br>
Recall from the slides that *DistilBERT* was created using a technique called *knowledge distillation*. The result is a model that performs almost as well as BERT but is 40% smaller and 60% faster.<br>
DistilBert Paper: https://arxiv.org/abs/1910.01108<br>
https://en.wikipedia.org/wiki/Knowledge_distillation
<br><br>
*distilroberta-base* was created by applying knowledge distillation to *Roberta-Base*, a more powerful model than BERT.<br>
Roberta paper: https://arxiv.org/abs/1907.11692<br><br>
The **Transformers** library provides a set of Auto Classes that can automatically retrieve configurations, tokenizers, and models based on a path or a name. We'll use the **AutoTokenizer** class to get the right tokenizer for *distilroberta-base*.<br>
https://huggingface.co/docs/transformers/main/en/model_doc/auto<br>
https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoTokenizer



In [79]:
model_name = 'distilroberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)

Calling *encode* converts a string to a sequence of integer token ids.<br>
https://huggingface.co/docs/transformers/main/en/internal/tokenization_utils#transformers.PreTrainedTokenizerBase.encode

In [80]:
t = "Where can I find a pizzeria?"
print(tokenizer.encode(t))

[0, 13841, 64, 38, 465, 10, 26432, 6971, 116, 2]


But to tokenize, we call the tokenizer object directly (i.e. using *\_\_call\_\_*).<br>
https://huggingface.co/docs/transformers/main/en/internal/tokenization_utils#transformers.PreTrainedTokenizerBase.__call__<br>

This returns a sequence of ids and an attention mask in a **BatchEncoding** object:<br>
https://huggingface.co/docs/transformers/main/en/glossary#input-ids<br>
https://huggingface.co/docs/transformers/main/en/main_classes/tokenizer#transformers.BatchEncoding<br>

Since there's no padding on this sample string, the mask is all 1s.

In [81]:
encoded_t = tokenizer(t)
print(encoded_t)

{'input_ids': [0, 13841, 64, 38, 465, 10, 26432, 6971, 116, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


We can convert the ids back to tokens using *convert_ids_to_tokens*.<br>
https://huggingface.co/docs/transformers/main/en/main_classes/tokenizer#transformers.PreTrainedTokenizer.convert_ids_to_tokens<br>

Note how the tokenizer added a start of sequence token (\<s\>), end of sequence token (\</s\>), and how it uses Ġ to signal a word has preceding whitespace. Keep in mind that what you're seeing here is the output from the *distilroberta-base* tokenizer. Other tokenizers may work differently.

In [82]:
print(tokenizer.convert_ids_to_tokens(encoded_t['input_ids']))

['<s>', 'Where', 'Ġcan', 'ĠI', 'Ġfind', 'Ġa', 'Ġpizz', 'eria', '?', '</s>']


As we covered in the slides, for question answering, we need to encode the question and context as a pair. In our case, we can do that by passing in both strings separated by a comma.

In [83]:
encoded_pair = tokenizer("this is a question", "this is the context")
print(encoded_pair)

{'input_ids': [0, 9226, 16, 10, 864, 2, 2, 9226, 16, 5, 5377, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


The *distilroberta-base* tokenizer uses a double \</s\>\</s\> as a separator.

In [84]:
print(tokenizer.convert_ids_to_tokens(encoded_pair['input_ids']))

['<s>', 'this', 'Ġis', 'Ġa', 'Ġquestion', '</s>', '</s>', 'this', 'Ġis', 'Ġthe', 'Ġcontext', '</s>']


**Side note**:<br>
Most of the tokenizers in the **Transformers** library come in two versions: a Python implementation and a faster Rust implementation. When available, **Autotokenizer** will download the fast version.<br>
https://huggingface.co/docs/transformers/main_classes/tokenizer<br>

We can check whether we have a fast tokenizer.


In [85]:
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

Suppose we tokenize this question/context pair...

In [86]:
context = "Sarah went to The Mirthless Cafe last night to meet her friend."
question = "Where did Sarah go?"

# The answer span and the answer's starting character position in the context.
answer = "The Mirthless Cafe"
answer_start = 14

In [87]:
x = tokenizer(question, context)
x

{'input_ids': [0, 13841, 222, 4143, 213, 116, 2, 2, 33671, 439, 7, 20, 256, 24208, 1672, 16542, 94, 363, 7, 972, 69, 1441, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Note how the word *Mirthless* gets tokenized into subwords. For legibility, we're using *batch_decode* to convert the input_ids to strings.<br>
https://huggingface.co/docs/transformers/v4.23.1/en/internal/tokenization_utils#transformers.PreTrainedTokenizerBase.batch_decode

In [88]:
tokenizer.batch_decode(x['input_ids'])

['<s>',
 'Where',
 ' did',
 ' Sarah',
 ' go',
 '?',
 '</s>',
 '</s>',
 'Sarah',
 ' went',
 ' to',
 ' The',
 ' M',
 'irth',
 'less',
 ' Cafe',
 ' last',
 ' night',
 ' to',
 ' meet',
 ' her',
 ' friend',
 '.',
 '</s>']

When we tokenize our dataset, there will probably be question/context pairs which exceed our model's maximum sequence length. In *Roberta*'s case, that's 512. Available GPU memory may make us further reduce the maximum sequence length of our input.<br>

Let's say the maximum sequence length we can handle is 15, so we truncate the context.

In [89]:
example_max_length = 15
x = tokenizer(question, context, max_length=example_max_length, 
              truncation="only_second")
x

{'input_ids': [0, 13841, 222, 4143, 213, 116, 2, 2, 33671, 439, 7, 20, 256, 24208, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

The problem here is that the answer span gets chopped off by truncation. In other situations, the answer may not be included at all.

In [90]:
tokenizer.batch_decode(x['input_ids'])

['<s>',
 'Where',
 ' did',
 ' Sarah',
 ' go',
 '?',
 '</s>',
 '</s>',
 'Sarah',
 ' went',
 ' to',
 ' The',
 ' M',
 'irth',
 '</s>']

To ensure we tokenize all context tokens while respecting a maximum length, we can set *return_overflowing_tokens* to **True**. The end effect is to split the input into multiple question/context sequences, with each context sequence being a continuation of the previous one. Since the last one may be shorter than the max length, we set the right padding length as well.<br>

What we get back are multiple *input_id* sequences.

In [91]:
x = tokenizer(question, context, max_length=example_max_length, 
              truncation="only_second", return_overflowing_tokens=True, 
              padding="max_length")
x

{'input_ids': [[0, 13841, 222, 4143, 213, 116, 2, 2, 33671, 439, 7, 20, 256, 24208, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 1672, 16542, 94, 363, 7, 972, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 69, 1441, 4, 2, 1, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]], 'overflow_to_sample_mapping': [0, 0, 0]}

In [93]:
len(x['input_ids'])

3

Looking at the decoded sequences, we see the entire context is included across three sequences (along with padding on the last one).<br>

In [94]:
tokenizer.batch_decode(x['input_ids'])

['<s>Where did Sarah go?</s></s>Sarah went to The Mirth</s>',
 '<s>Where did Sarah go?</s></s>less Cafe last night to meet</s>',
 '<s>Where did Sarah go?</s></s> her friend.</s><pad><pad><pad>']

Note a few things from the encoded object *x*:
- The last *attention_mask* sequence has 0s to signify padding.
- The *overflow_to_sample_mapping* array tells us which question/context pair each *input_ids* sequence comes from. In our example, we tokenized a single question/context pair which resulted in three *input_ids* sequences, so *overflow_to_sample_mapping* is 3 0s.<br>

If we tokenize two question/context pairs, we'll see the *overflow_to_sample_mapping* reflect that.

In [95]:
tokenizer(['question 1', 'question 2'], 
          ['context 1', 'context 2'], 
          return_overflowing_tokens=True)

{'input_ids': [[0, 40018, 112, 2, 2, 46796, 112, 2], [0, 40018, 132, 2, 2, 46796, 132, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]], 'overflow_to_sample_mapping': [0, 1]}

But there's still a problem here in that none of the sequences contain the full answer ("The Mirthless Cafe"). Right now, the correct full answer is split across sequences.<br>

To counter this, we can tokenize our question/context pair into overlapping sequences by setting a *stride* length. We did something similar when we prepared the dataset for our [character-level language model](https://colab.research.google.com/github/nitinpunjabi/nlp-demystified/blob/main/notebooks/nlpdemystified_recurrent_neural_networks.ipynb#scrollTo=X1c-ihOByy88).

In [96]:
stride = 5
x = tokenizer(question, context, max_length=example_max_length, 
              truncation="only_second", return_overflowing_tokens=True,
              stride=stride, padding="max_length")

By setting a stride of 5, each context sequence starts 5 subwords back from the previous sequence.<br>

This way, two of our tokenized sequences now contain the full answer.


In [97]:
tokenizer.batch_decode(x['input_ids'])

['<s>Where did Sarah go?</s></s>Sarah went to The Mirth</s>',
 '<s>Where did Sarah go?</s></s> went to The Mirthless</s>',
 '<s>Where did Sarah go?</s></s> to The Mirthless Cafe</s>',
 '<s>Where did Sarah go?</s></s> The Mirthless Cafe last</s>',
 '<s>Where did Sarah go?</s></s> Mirthless Cafe last night</s>',
 '<s>Where did Sarah go?</s></s>irthless Cafe last night to</s>',
 '<s>Where did Sarah go?</s></s>less Cafe last night to meet</s>',
 '<s>Where did Sarah go?</s></s> Cafe last night to meet her</s>',
 '<s>Where did Sarah go?</s></s> last night to meet her friend</s>',
 '<s>Where did Sarah go?</s></s> night to meet her friend.</s>']

We now have a way to tokenize our question/context pairs.<br>

Our tokenizer returned this **BatchEncoding** object:

In [98]:
print(x.keys(), '\n')
x

dict_keys(['input_ids', 'attention_mask', 'overflow_to_sample_mapping']) 



{'input_ids': [[0, 13841, 222, 4143, 213, 116, 2, 2, 33671, 439, 7, 20, 256, 24208, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 439, 7, 20, 256, 24208, 1672, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 7, 20, 256, 24208, 1672, 16542, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 20, 256, 24208, 1672, 16542, 94, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 256, 24208, 1672, 16542, 94, 363, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 24208, 1672, 16542, 94, 363, 7, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 1672, 16542, 94, 363, 7, 972, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 16542, 94, 363, 7, 972, 69, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 94, 363, 7, 972, 69, 1441, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 363, 7, 972, 69, 1441, 4, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 

To fine-tune a model for question answering, our pre-trained *distilroberta-base* model expects this object to contain two more pieces of information:
- *start_positions*: the token positions where answers begin.
- *end_positions*: the token positions where answers end.<br>

https://huggingface.co/docs/transformers/main/en/model_doc/roberta#transformers.RobertaForQuestionAnswering.forward

All we have in our example (and the SQuAD dataset) is the position of the starting  <u>character</u> of the answer.

In [99]:
print(answer_start)
print(context[answer_start:answer_start+len(answer)])

14
The Mirthless Cafe


We need to use this to locate the <u>token</u> positions where each answer starts and ends in every *input_ids* sequence. In some cases, the complete answer may not be in a particular sequence. We need to handle those cases as well.<br>

To do this, we'll get more information by setting *return_offsets_mapping* to **True** in the tokenizer.

In [100]:
x = tokenizer(question, context, max_length=example_max_length, 
              truncation="only_second", return_overflowing_tokens=True,
              stride=stride, return_offsets_mapping=True,
              padding="max_length")
x

{'input_ids': [[0, 13841, 222, 4143, 213, 116, 2, 2, 33671, 439, 7, 20, 256, 24208, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 439, 7, 20, 256, 24208, 1672, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 7, 20, 256, 24208, 1672, 16542, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 20, 256, 24208, 1672, 16542, 94, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 256, 24208, 1672, 16542, 94, 363, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 24208, 1672, 16542, 94, 363, 7, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 1672, 16542, 94, 363, 7, 972, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 16542, 94, 363, 7, 972, 69, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 94, 363, 7, 972, 69, 1441, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 363, 7, 972, 69, 1441, 4, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 

This results in *offset_mapping* sequences, one for each *input_ids* sequence. 

In [101]:
print(len(x['input_ids']))
print(len(x['offset_mapping']))

10
10


Each entry in an *offset_mapping* tells us the starting and ending character position of each token in the original string. An offset mapping of (0,0) represents a special token (e.g. \<s\>).<br>

For example, here's the first *input_ids* sequence along with its respective *offset_mapping*.

In [102]:
print(x['input_ids'][0])
print(x['offset_mapping'][0])

[0, 13841, 222, 4143, 213, 116, 2, 2, 33671, 439, 7, 20, 256, 24208, 2]
[(0, 0), (0, 5), (6, 9), (10, 15), (16, 18), (18, 19), (0, 0), (0, 0), (0, 5), (6, 10), (11, 13), (14, 17), (18, 19), (19, 23), (0, 0)]


If we convert the first non-special input id to a token, and use the first non-special offset_mapping to extract a span from the question string, we get a match.

In [103]:
print("First non-special input_id converted to token:")
print(tokenizer.convert_ids_to_tokens(x['input_ids'][0][1]), "\n")

offset = x['offset_mapping'][0][1]
print(f"Span extracted from context using corresponding offset_mapping {offset}:")
print(question[offset[0]:offset[1]])

First non-special input_id converted to token:
Where 

Span extracted from context using corresponding offset_mapping (0, 5):
Where


Since we know the character position of where the answer starts, we can use that and *offset_mapping* to get the start and ending token positions of the answer span.<br>

The only remaining issue is identifying whether an offset is for a question or a context. Looking at the first two *offset_mappings*, note that:<br>
1. In the first sequence, both the question and context *offset_mappings* start from zero.
2. In the second sequence, the context *offset_mapping* values carry on from the previous sequence (after accounting in the stride).

In [104]:
print(x['offset_mapping'][0])
print(x['offset_mapping'][1])

[(0, 0), (0, 5), (6, 9), (10, 15), (16, 18), (18, 19), (0, 0), (0, 0), (0, 5), (6, 10), (11, 13), (14, 17), (18, 19), (19, 23), (0, 0)]
[(0, 0), (0, 5), (6, 9), (10, 15), (16, 18), (18, 19), (0, 0), (0, 0), (6, 10), (11, 13), (14, 17), (18, 19), (19, 23), (23, 27), (0, 0)]


This means we need to identify
1. which *offset_mappings* belong to a context.
2. whether a particular sequence contains the answer at all.<br>

The first can be done using the *sequence_ids* method on the encoding object. Each *input_ids* sequence has a corresponding *sequence_ids* list which tells us whether a token is part of a question, part of a context, or a special token.<br>
https://huggingface.co/docs/transformers/main/en/main_classes/tokenizer#transformers.BatchEncoding.sequence_ids

In [105]:
print(x['input_ids'][0])
print(x.sequence_ids(0))

[0, 13841, 222, 4143, 213, 116, 2, 2, 33671, 439, 7, 20, 256, 24208, 2]
[None, 0, 0, 0, 0, 0, None, None, 1, 1, 1, 1, 1, 1, None]


So to identify whether a token is part a context, we can use *sequence_ids* to check whether a token position maps to 1.

For the second issue, we can check whether the answer start and end character positions are within the lowest and highest offset mapping values respectively.

In [106]:
# We can calculate the answer end character position using the answer length.
answer_end = answer_start + len(answer)

print("Answer start character position:", answer_start)
print("Answer end character position:", answer_end)
print("Answer pulled from context:", context[answer_start:answer_end])

Answer start character position: 14
Answer end character position: 32
Answer pulled from context: The Mirthless Cafe


Let's find the start and end token positions from our collection of sequences. The full answer is not in the first sequence, but is in the third sequence. So let's experiment with those.

In [107]:
tokenizer.batch_decode(x['input_ids'])

['<s>Where did Sarah go?</s></s>Sarah went to The Mirth</s>',
 '<s>Where did Sarah go?</s></s> went to The Mirthless</s>',
 '<s>Where did Sarah go?</s></s> to The Mirthless Cafe</s>',
 '<s>Where did Sarah go?</s></s> The Mirthless Cafe last</s>',
 '<s>Where did Sarah go?</s></s> Mirthless Cafe last night</s>',
 '<s>Where did Sarah go?</s></s>irthless Cafe last night to</s>',
 '<s>Where did Sarah go?</s></s>less Cafe last night to meet</s>',
 '<s>Where did Sarah go?</s></s> Cafe last night to meet her</s>',
 '<s>Where did Sarah go?</s></s> last night to meet her friend</s>',
 '<s>Where did Sarah go?</s></s> night to meet her friend.</s>']

First get all the information we need for the first sequence.

In [108]:
input_ids = x['input_ids'][0]
offset_mapping = x['offset_mapping'][0]
seq_ids = x.sequence_ids(0)

Determine where the context tokens start and end in the sequence.

In [109]:
# These are the sequence ids
print("Sequence IDs: ", seq_ids)

Sequence IDs:  [None, 0, 0, 0, 0, 0, None, None, 1, 1, 1, 1, 1, 1, None]


In [110]:
# Get the start index position (i.e. the first occurrence of 1).
context_pos_start = seq_ids.index(1)

In [111]:
# Utility function to find the *last* occurrence of a sequence.
def rindex(lst, value):
    return len(lst) - operator.indexOf(reversed(lst), value) - 1

# Get the end index position (i.e. the last occurrence of 1).
context_pos_end = rindex(seq_ids, 1)

In [112]:
print("Context tokens begin at position", context_pos_start)
print("Context tokens end at position", context_pos_end)

Context tokens begin at position 8
Context tokens end at position 13


Now that we know which tokens are part of the context, we can look at their corresponding offset mappings to check whether the start and end character positions are within the offsets.

In [113]:
# These are the corresponding offsets.
context_offsets = offset_mapping[context_pos_start:context_pos_end+1]
print(context_offsets)

[(0, 5), (6, 10), (11, 13), (14, 17), (18, 19), (19, 23)]


In [115]:
print("Is the lowest offset value lower than or equal to the starting character position?")
print("Answer starting character position:", answer_start)
print("First offset:", context_offsets[0])

# Note how we're checking the first tuple value.
print(context_offsets[0][0] <= answer_start)

Is the lowest offset value lower than or equal to the starting character position?
Answer starting character position: 14
First offset: (0, 5)
True


In [116]:
print("Is the highest offset value higher than or equal to the ending character position?")
print("Answer ending character position:", answer_end)
print("Last offset:", context_offsets[-1])

# Note how how we're checking the second tuple value.
print(context_offsets[-1][1] >= answer_end)

Is the highest offset value higher than or equal to the ending character position?
Answer ending character position: 32
Last offset: (19, 23)
False


So the first sequence contains a part of the answer but the full answer gets truncated. This matches a visual inspection:

In [117]:
print(tokenizer.batch_decode(input_ids))

['<s>', 'Where', ' did', ' Sarah', ' go', '?', '</s>', '</s>', 'Sarah', ' went', ' to', ' The', ' M', 'irth', '</s>']


Let's now do the same with the third sequence.

In [118]:
input_ids = x['input_ids'][2]
offset_mapping = x['offset_mapping'][2]
seq_ids = x.sequence_ids(2)

context_pos_start = seq_ids.index(1)
context_pos_end = rindex(seq_ids, 1)

context_offsets = offset_mapping[context_pos_start:context_pos_end+1]

print("Is the lowest offset value lower than or equal to the starting character position?")
print("Answer starting character position:", answer_start)
print("First offset:", context_offsets[0])

# Note how we're checking the first tuple value.
print(context_offsets[0][0] <= answer_start)

print("Is the highest offset value higher than or equal to the ending character position?")
print("Answer ending character position:", answer_end)
print("Last offset:", context_offsets[-1])

# Note how how we're checking the second tuple value.
print(context_offsets[-1][1] >= answer_end)


Is the lowest offset value lower than or equal to the starting character position?
Answer starting character position: 14
First offset: (11, 13)
True
Is the highest offset value higher than or equal to the ending character position?
Answer ending character position: 32
Last offset: (28, 32)
True


Now that we've confirmed the third sequence contains the full answer, we need to identify where the answer starts and ends in the *input_ids*. We can do this by scanning the offset_mapping from the left to find the start, and from the right to find the end.

In [119]:
s = e = 0

# Start scanning the offset_mapping from the
# left to find the token position where the answer starts.
# It's not guaranteed a tokenizer will output a token where the
# starting character matches the first answer character. When
# this happens, we take the previous token's position as our start.
i = context_pos_start
while offset_mapping[i][0] < answer_start:
  i += 1
if offset_mapping[i][0] == answer_start:
  s = i
else:
  s = i - 1

# Same idea when finding the ending token position.
j = context_pos_end
while offset_mapping[j][1] > answer_end:
  j -= 1      
if offset_mapping[j][1] == answer_end:
  e = j
else:
  e = j + 1

In [120]:
print("Answer start token position in context:", s)
print("Answer end token position in context:", e)

Answer start token position in context: 9
Answer end token position in context: 13


In [121]:
print("Answer lifted from context:")
tokenizer.batch_decode(input_ids[s:e+1])

Answer lifted from context:


[' The', ' M', 'irth', 'less', ' Cafe']

All the logic we stepped through so far is encapsulated in the following method. We'll use this to process our dataset.

In [122]:
def prepare_dataset(examples):
  # Some tokenizers don't strip spaces. If there happens to be question text
  # with excessive spaces, the context may not get encoded at all.
  examples["question"] = [q.lstrip() for q in examples["question"]]
  examples["context"] = [c.lstrip() for c in examples["context"]]

  # Tokenize.  
  tokenized_examples = tokenizer(
      examples['question'],
      examples['context'],
      truncation="only_second",
      max_length = max_length,
      stride=stride,
      return_overflowing_tokens=True,
      return_offsets_mapping=True,
      padding="max_length"
  )

  # We'll collect a list of starting positions and ending positions.
  tokenized_examples['start_positions'] = []
  tokenized_examples['end_positions'] = []

  # Work through every sequence.
  for seq_idx in range(len(tokenized_examples['input_ids'])):
    seq_ids = tokenized_examples.sequence_ids(seq_idx)
    offset_mappings = tokenized_examples['offset_mapping'][seq_idx]

    cur_example_idx = tokenized_examples['overflow_to_sample_mapping'][seq_idx]
    answer = examples['answers'][cur_example_idx]
    answer_text = answer['text'][0]
    answer_start = answer['answer_start'][0]
    answer_end = answer_start + len(answer_text)

    context_pos_start = seq_ids.index(1)
    context_pos_end = rindex(seq_ids, 1)

    s = e = 0
    if (offset_mappings[context_pos_start][0] <= answer_start and
        offset_mappings[context_pos_end][1] >= answer_end):
      i = context_pos_start
      while offset_mappings[i][0] < answer_start:
        i += 1
      if offset_mappings[i][0] == answer_start:
        s = i
      else:
        s = i - 1

      j = context_pos_end
      while offset_mappings[j][1] > answer_end:
        j -= 1      
      if offset_mappings[j][1] == answer_end:
        e = j
      else:
        e = j + 1

    tokenized_examples['start_positions'].append(s)
    tokenized_examples['end_positions'].append(e)

  return tokenized_examples

Before we process, we'll set maximum sequence length, stride, and batch size values.<br>

I arrived at these values through experimentation. Even though *distilroberta-base* has a maximum sequence length of 512, using the full capacity (or a large batch value) results in an out-of-memory error while the attention scores are being calculated. This is on Colab's free tier. On the premium tier, you can use larger sequence lengths or batch values.<br>

The nature of the data will also influence the values.

In [123]:
max_length = 400
stride = 100
batch_size = 32

We can map over the **Dataset** objects and apply our prepare method to the examples in batches.<br>
https://huggingface.co/docs/datasets/main/en/nlp_process<br>
https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.map<br>

*remove_columns* removes the original data columns and leaves only the post-tokenization columns in place. We can also parallelize processing by using the *num_proc* parameter.<br>
https://huggingface.co/docs/datasets/main/en/process#multiprocessing

In [124]:
tokenized_datasets = data.map(
  prepare_dataset,
  batched=True,
  remove_columns=data["train"].column_names,
  num_proc=2,
)

Map (num_proc=2):   0%|          | 0/87599 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/10570 [00:00<?, ? examples/s]

Our tokenized dataset still contains two entries (*offset_mapping* and *overflow_to_sample_mapping*) our model won't expect, so we'll remove them.<br>
https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.remove_columns

In [125]:
data = tokenized_datasets.remove_columns(["offset_mapping", 
                                          "overflow_to_sample_mapping"])

The last preparation step is to convert the Hugging Face **Dataset** objects into a Tensorflow-compatible datasets.<br>
https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.to_tf_dataset<br>
https://huggingface.co/docs/datasets/main/en/use_with_tensorflow#when-to-use-totfdataset

In [126]:
train_set = data['train'].to_tf_dataset(batch_size=batch_size)
validation_set = data['validation'].to_tf_dataset(batch_size=batch_size)

We can now download a pre-trained model for fine-tuning. Just like we did with the tokenizer, we'll use an Auto Class to download the right model. In this case, we're using **TFAutoModelForQuestionAnswering**. This will download a Tensorflow implementation of the pre-trained model with a question answering head on it.<br>

The head in this case is a dense layer that returns *start_logits* and *end_logits*. We can take the argmax of each to determine the start and end of the answer span (see model code for details).<br>
https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.TFAutoModelForQuestionAnswering<br>
https://github.com/huggingface/transformers/blob/main/src/transformers/models/roberta/modeling_tf_roberta.py#L1629

In [127]:
model = TFAutoModelForQuestionAnswering.from_pretrained(model_name)

All model checkpoint layers were used when initializing TFRobertaForQuestionAnswering.

Some layers of TFRobertaForQuestionAnswering were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The following method attempts to answer a question given a context. It tokenizes the question and context, runs it through the model, takes the argmax of the start and end logits, and uses the result to extract an answer span from the context.

In [128]:
def get_answer(tokenizer, model, question, context):
  inputs = tokenizer([question], [context], return_tensors="np")
  outputs = model(inputs)
  start_position = tf.argmax(outputs.start_logits, axis=1)
  end_position = tf.argmax(outputs.end_logits, axis=1)
  answer = inputs["input_ids"][0, int(start_position) : int(end_position) + 1]
  return tokenizer.decode(answer).strip()

While the model body (the pre-trained *distilroberta-base* model) is trained, the head is not. So if we try to use our model to answer a question, it should fail or perform poorly (your output will differ because of different initial head weight values).

In [129]:
c = "Sarah went to The Mirthless Cafe last night to meet her friend."
q = "Where did Sarah go?"
get_answer(tokenizer, model, q, c)

''

In [130]:
# https://www.tensorflow.org/guide/mixed_precision
keras.mixed_precision.set_global_policy("mixed_float16")

# Use a learning rate recommended by the BERT authors.
# https://github.com/google-research/bert
model.compile(optimizer=keras.optimizers.Adam(learning_rate=3e-5))

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


We'll now fine-tune the model. Note that we didn't freeze the layers of the pre-trained body, so its weights will be tuned along with the head's weights.<br>

Because the body is already pre-trained, we don't need a lot of epochs. 2-4 is typically enough (BERT authors recommend 4). Here, we're using 1 to demonstrate the power of pre-training.<br>

**Note:** If you have GPU enabled and you're using Colab's free tier, the training time can be all over the place depending on which GPU you get assigned (anywhere from 20 minutes to an hour).

In [ ]:
model.fit(train_set, validation_data=validation_set, epochs=1)

 457/2761 [===>..........................] - ETA: 55:32 - loss: 2.0026

After completing our fine-tuning, we should now have a decent extractive question answering model.

In [ ]:
c = "Sarah went to The Mirthless Cafe last night to meet her friend."
q = "Where did Sarah go?"
get_answer(tokenizer, model, q, c)

In [ ]:
q = "Who did Sarah meet?"
get_answer(tokenizer, model, q, c)

In [ ]:
q = "When did Sarah meet her friend?"
get_answer(tokenizer, model, q, c)

In [ ]:
q = "Who went to the restaurant?"
get_answer(tokenizer, model, q, c)

But as we saw earlier, extractive question answering has its limits.

In [ ]:
# Asking a logic teaser question is difficult despite the
# answer being available. To be fair, there is ambiguity here.
q = "Who did Sarah's friend meet?"
get_answer(tokenizer, model, q, c)

In [ ]:
# The model can't determine when a question can't be 
# answered. Some question answering datasets explicitly 
# train for this.
q = "How did Sarah get to the restaurant?"
get_answer(tokenizer, model, q, c)

In [ ]:
# The model isn't generative, either.
q = "What is a possible reason for why Sarah met her friend?"
get_answer(tokenizer, model, q, c)

But despite this model's limitations, I hope this shows the power of pre-training and how fast you can get something cool and useful up and running.<br>

I encourage you to make an account on Hugging Face and push your model to the hub. Learn how here:
https://huggingface.co/docs/transformers/model_sharing


We're just scratching the surface of question answering. Indeed you could dedicate an entire career to it. Areas to explore:
- Right now, we have to supply the context along with the question. A more sophisticated system would load all relevant documents into some database and search over it for an appropriate context/passage and then extract the answer from it. If multiple answers are extracted, then maybe some ranking system can be included as well.
- Another enhancement is extracting answers from different kinds of data. Beyond text, there are images, audio, graphs, tables, charts, etc.
- Abstractive answering involves composing answers (possibly multiple lines) rather than extracting them. Open book means having the system search for the answer first in a database, then composing an answer based on what it's found. Closed book means the model relies on its internal knowledge only. This is what a large language model like GPT-3 would do.
<br>

Speaking of GPT-3, let's play with a few more prompts. At this point, we'll switch to the [OpenAI Playground](https://beta.openai.com/playground) and try out the prompts below. Check out the module video for commentary.<br>

Before you can try out these prompts yourself, you'll need to open an account an [OpenAI](https://openai.com/api/). At the time of this recording, OpenAI was providing a few dollars of credit to get started. It's more than enough to run the prompts below.

## GPT-3 prompts to try out:

---
He said hello<br>
She said bonjour<br>
He said goodbye<br>
She said<br>

---
My name is Kilgore. I run a revenge-for-hire business called Delightful Reprisals. I billed my customer the wrong amount. Write an apology email to the customer and sign it with my name and company. Include a reason blaming Elon Musk.

---
He brought three pies to the office. He gave one to his co-workers, and threw one at his boss' face. How many pies did he have left?

---
Answer in the style of Jeopardy. He was the 26th President of the United States.

---
Sarah went to The Mirthless Cafe last night to meet her friend.<br>

Print each answer to the following questions on separate lines.<br>

Where did Sarah go?<br>
Who did Sarah meet?<br>
When did Sarah meet her friend?<br>
How did Sarah get to the restaurant?<br>
What's a possible reason why they met?<br>

---
What did Marie Antoinette say about avocado toast?

---
What's a word that rhymes with money?

---
What's a word that rhymes with Kafkaesque?

---
Write a haiku about The Terminator.

---
Write a limerick about The Terminator.

---

There are open-source generative decoder models available on Hugging Face as well. Among them are:
- [GPT-2](https://huggingface.co/gpt2)
- [EleutherAI/gpt-neo-1.3B](https://huggingface.co/EleutherAI/gpt-neo-1.3B)
- [bigscience/bloom-560m](https://huggingface.co/bigscience/bloom-560m)


**Be on the lookout for GPT-4!**

# Further Exploration

OpenAI API docs to learn how to build products using their models:<br>
https://openai.com/api/<br>
https://beta.openai.com/docs/introduction

A catalog of transformer models:<br>
https://amatriain.net/blog/transformer-models-an-introduction-and-catalog-2d1e9039f376/


Wordpiece and Sentencepiece:<br>
https://huggingface.co/course/chapter6/6?fw=pt<br>
https://github.com/google/sentencepiece


**Papers**<br>
Attention Is All You Need (original Transformer paper): https://arxiv.org/abs/1706.03762<br>

The Annotated Transformer: http://nlp.seas.harvard.edu/annotated-transformer/<br>

GPT-3: https://arxiv.org/abs/2005.14165<br>

BERT: https://arxiv.org/abs/1810.04805<br>

RoBERTa paper: https://arxiv.org/abs/1907.11692<br>

ALBERT paper: https://arxiv.org/abs/1909.11942<br>

DistilBert paper: https://arxiv.org/abs/1910.01108<br>

Electra paper: https://arxiv.org/abs/2003.10555<br>

XLM: https://arxiv.org/abs/1901.07291<br>